<a href="https://colab.research.google.com/github/JoshuaCarriaga/NFL-Fantasy-23/blob/main/Fantasy_23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import lxml
import html5lib


#WEBSITE TO PYTHON ATTEMPT (LINE BY LINE)
#url='https://www.pro-football-reference.com/years/2023/fantasy.htm'
#html=requests.get(url).content
#df_list=pd.read_html(html)
#df=df_list[0]
#df.to_csv("23_fantasy_points")

# RESULTS HAD MULTIPLE LINES I DIDN'T LIKE SO I IMPORTED TO GOOGLE SHEETS (AGAINST MY WILL) AND HAD TO REMOVE THE
# LINES THAT CREATED BREAKS IN MY DATA



# Importing data set
Looking through the data to see what needs to be cleaned

In [3]:
#WEBSITE TO PYTHON ATTEMPT (ONE LINE)
fp = pd.read_html(requests.get("https://www.pro-football-reference.com/years/2023/fantasy.htm").content)[0]
fp.to_csv('fp')



In [4]:
fp.describe()

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0  \
                       Rk             Player                 Tm   
count                 653                653                653   
unique                633                633                 35   
top                    Rk             Player                LAC   
freq                   21                 21                 23   

       Unnamed: 3_level_0 Unnamed: 4_level_0 Games      Passing            \
                  FantPos                Age     G   GS     Cmp  Att  Yds   
count                 653                653   653  653     653  653  653   
unique                  6                 21    18   19      67   74   85   
top                    WR                 24    17    0       0    0    0   
freq                  243                100   150  216     545  534  545   

        ... Scoring          Fantasy                                         
        ...      TD 2PM  2PP  FantPt  PPR  DKPt  FDPt  VBD  PosRank  OvRank  
count   ...     653  79   49     549  564   564   563   96      653      99  
unique  ...      18   4    4     180  449   457   443   51      244      79  
top     ...       0   1  2PP       1  PPR  DKPt  FDPt  VBD  PosRank  OvRank  
freq    ...     296  50   21      25   21    21    21   21       21      21  

[4 rows x 33 columns]

In [5]:
fp.head()

Unnamed: 0_level_0     Unnamed: 1_level_0 Unnamed: 2_level_0  \
                  Rk                 Player                 Tm   
0                  1  Christian McCaffrey*+                SFO   
1                  2          CeeDee Lamb*+                DAL   
2                  3             Josh Allen                BUF   
3                  4          Tyreek Hill*+                MIA   
4                  5           Jalen Hurts*                PHI   

  Unnamed: 3_level_0 Unnamed: 4_level_0 Games     Passing             ...  \
             FantPos                Age     G  GS     Cmp  Att   Yds  ...   
0                 RB                 27    16  16       0    0     0  ...   
1                 WR                 24    17  17       0    0     0  ...   
2                 QB                 27    17  17     385  579  4306  ...   
3                 WR                 29    16  16       0    0     0  ...   
4                 QB                 25    17  17     352  538  3858  ...   

  Scoring           Fantasy                                           
       TD  2PM  2PP  FantPt    PPR   DKPt   FDPt  VBD PosRank OvRank  
0      21  NaN  NaN     324  391.3  399.3  357.8  157       1      1  
1      14    1  NaN     268  403.2  411.2  335.7  131       1      2  
2      15  NaN    3     393  392.6  420.6  410.6  122       1      3  
3      13  NaN  NaN     257  376.4  380.4  316.9  120       2      4  
4      15  NaN  NaN     357  356.8  382.8  371.8   89       2      5  

[5 rows x 33 columns]

In [6]:
fp_cols=("Rk","Player","Tm","FantPos","Age","G","GS","Cmp","PassAtt","PassYds","PassTD","Int","RushAtt","RushYds","Y/A","RushTD","Tgt","Rec","RecYds","Y/R","RecTD","Fmb","FL","TotalTD","2PM","2PP","FantPt","PPR","DKPt","FDPt","VBD","PosRank","OvRank")

In [7]:
fp.columns = fp_cols

In [8]:
fp.head()

,Rk,Player,Tm,FantPos,Age,G,GS,Cmp,PassAtt,PassYds,...,TotalTD,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank
0,1,Christian McCaffrey*+,SFO,RB,27,16,16,0,0,0,...,21,NaN,NaN,324,391.3,399.3,357.8,157,1,1
1,2,CeeDee Lamb*+,DAL,WR,24,17,17,0,0,0,...,14,1,NaN,268,403.2,411.2,335.7,131,1,2
2,3,Josh Allen,BUF,QB,27,17,17,385,579,4306,...,15,NaN,3,393,392.6,420.6,410.6,122,1,3
3,4,Tyreek Hill*+,MIA,WR,29,16,16,0,0,0,...,13,NaN,NaN,257,376.4,380.4,316.9,120,2,4
4,5,Jalen Hurts*,PHI,QB,25,17,17,352,538,3858,...,15,NaN,NaN,357,356.8,382.8,371.8,89,2,5


In [9]:
fp.describe()

,Rk,Player,Tm,FantPos,Age,G,GS,Cmp,PassAtt,PassYds,...,TotalTD,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank
count,653,653,653,653,653,653,653,653,653,653,...,653,79,49,549,564,564,563,96,653,99
unique,633,633,35,6,21,18,19,67,74,85,...,18,4,4,180,449,457,443,51,244,79
top,Rk,Player,LAC,WR,24,17,0,0,0,0,...,0,1,2PP,1,PPR,DKPt,FDPt,VBD,PosRank,OvRank
freq,21,21,23,243,100,150,216,545,534,545,...,296,50,21,25,21,21,21,21,21,21


## Realizing that the column names are repeated in the rows

In [10]:
#fp['Age'].median()

#TypeError - Cannot Convert
## There are instances of non-integers in the Age Column

In [11]:
fp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 653 entries, 0 to 652
Data columns (total 33 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Rk       653 non-null    object
 1   Player   653 non-null    object
 2   Tm       653 non-null    object
 3   FantPos  653 non-null    object
 4   Age      653 non-null    object
 5   G        653 non-null    object
 6   GS       653 non-null    object
 7   Cmp      653 non-null    object
 8   PassAtt  653 non-null    object
 9   PassYds  653 non-null    object
 10  PassTD   653 non-null    object
 11  Int      653 non-null    object
 12  RushAtt  653 non-null    object
 13  RushYds  653 non-null    object
 14  Y/A      354 non-null    object
 15  RushTD   653 non-null    object
 16  Tgt      653 non-null    object
 17  Rec      653 non-null    object
 18  RecYds   653 non-null    object
 19  Y/R      477 non-null    object
 20  RecTD    653 non-null    object
 21  Fmb      653 non-null    object
 22  FL

In [12]:
fp.fillna("0")

,Rk,Player,Tm,FantPos,Age,G,GS,Cmp,PassAtt,PassYds,...,TotalTD,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank
0,1,Christian McCaffrey*+,SFO,RB,27,16,16,0,0,0,...,21,0,0,324,391.3,399.3,357.8,157,1,1
1,2,CeeDee Lamb*+,DAL,WR,24,17,17,0,0,0,...,14,1,0,268,403.2,411.2,335.7,131,1,2
2,3,Josh Allen,BUF,QB,27,17,17,385,579,4306,...,15,0,3,393,392.6,420.6,410.6,122,1,3
3,4,Tyreek Hill*+,MIA,WR,29,16,16,0,0,0,...,13,0,0,257,376.4,380.4,316.9,120,2,4
4,5,Jalen Hurts*,PHI,QB,25,17,17,352,538,3858,...,15,0,0,357,356.8,382.8,371.8,89,2,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648,629,Deon Jackson,3TM,RB,24,4,1,0,0,0,...,0,0,0,-1,4.0,6.0,1.5,0,166,0
649,Rk,Player,Tm,FantPos,Age,G,GS,Cmp,Att,Yds,...,TD,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank
650,630,David Wells,TAM,TE,28,5,0,0,0,0,...,0,0,0,-1,1.0,1.0,0,0,140,0
651,631,James Proche,CLE,WR,27,10,1,0,0,0,...,0,0,0,-2,-2.0,-1.0,-2.0,0,242,0


In [13]:
fp.describe()
#noticed there is no median

,Rk,Player,Tm,FantPos,Age,G,GS,Cmp,PassAtt,PassYds,...,TotalTD,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank
count,653,653,653,653,653,653,653,653,653,653,...,653,79,49,549,564,564,563,96,653,99
unique,633,633,35,6,21,18,19,67,74,85,...,18,4,4,180,449,457,443,51,244,79
top,Rk,Player,LAC,WR,24,17,0,0,0,0,...,0,1,2PP,1,PPR,DKPt,FDPt,VBD,PosRank,OvRank
freq,21,21,23,243,100,150,216,545,534,545,...,296,50,21,25,21,21,21,21,21,21


In [14]:
#fp = fp.astype('Int64')
# ValueError: invalid literal for int() with base 10: 'Rk'
# Rk (and others) are in the columns and can't be converted to Int

In [15]:
#is_numeric = fp.str.isdecimal()
# AttributeError: 'DataFrame' object has no attribute 'str'


In [16]:
#for column in fp.columns:
 #   if fp[column].dtype == 'object':
  #    fp[column] = fp[column].astype('Int64')

#ValueError: invalid literal for int() with base 10: 'Rk'
# I realized I would need to remove the values that weren't numbers
# I'm sure there is a way to target just the numbers in the columns but
# I also believe that wouldn't fully clean the data

In [17]:
# Finding Rk in the dataframe (in Rk column)
rk = fp['Rk'].isin(['Rk'])

## Not the most efficient way to clean

In [18]:
fp[rk]
print(fp.head())

  Rk                 Player   Tm FantPos Age   G  GS  Cmp PassAtt PassYds  \
0  1  Christian McCaffrey*+  SFO      RB  27  16  16    0       0       0   
1  2          CeeDee Lamb*+  DAL      WR  24  17  17    0       0       0   
2  3             Josh Allen  BUF      QB  27  17  17  385     579    4306   
3  4          Tyreek Hill*+  MIA      WR  29  16  16    0       0       0   
4  5           Jalen Hurts*  PHI      QB  25  17  17  352     538    3858   

   ... TotalTD  2PM  2PP FantPt    PPR   DKPt   FDPt  VBD PosRank OvRank  
0  ...      21  NaN  NaN    324  391.3  399.3  357.8  157       1      1  
1  ...      14    1  NaN    268  403.2  411.2  335.7  131       1      2  
2  ...      15  NaN    3    393  392.6  420.6  410.6  122       1      3  
3  ...      13  NaN  NaN    257  376.4  380.4  316.9  120       2      4  
4  ...      15  NaN  NaN    357  356.8  382.8  371.8   89       2      5  

[5 rows x 33 columns]


In [19]:
print(fp[rk].head())

     Rk  Player  Tm  FantPos  Age  G  GS  Cmp PassAtt PassYds  ... TotalTD  \
29   Rk  Player  Tm  FantPos  Age  G  GS  Cmp     Att     Yds  ...      TD   
60   Rk  Player  Tm  FantPos  Age  G  GS  Cmp     Att     Yds  ...      TD   
91   Rk  Player  Tm  FantPos  Age  G  GS  Cmp     Att     Yds  ...      TD   
122  Rk  Player  Tm  FantPos  Age  G  GS  Cmp     Att     Yds  ...      TD   
153  Rk  Player  Tm  FantPos  Age  G  GS  Cmp     Att     Yds  ...      TD   

     2PM  2PP  FantPt  PPR  DKPt  FDPt  VBD  PosRank  OvRank  
29   2PM  2PP  FantPt  PPR  DKPt  FDPt  VBD  PosRank  OvRank  
60   2PM  2PP  FantPt  PPR  DKPt  FDPt  VBD  PosRank  OvRank  
91   2PM  2PP  FantPt  PPR  DKPt  FDPt  VBD  PosRank  OvRank  
122  2PM  2PP  FantPt  PPR  DKPt  FDPt  VBD  PosRank  OvRank  
153  2PM  2PP  FantPt  PPR  DKPt  FDPt  VBD  PosRank  OvRank  

[5 rows x 33 columns]


In [20]:
fp.drop([29,60,91,122,153], inplace=True)

In [21]:
print(fp[rk].head())

     Rk  Player  Tm  FantPos  Age  G  GS  Cmp PassAtt PassYds  ... TotalTD  \
184  Rk  Player  Tm  FantPos  Age  G  GS  Cmp     Att     Yds  ...      TD   
215  Rk  Player  Tm  FantPos  Age  G  GS  Cmp     Att     Yds  ...      TD   
246  Rk  Player  Tm  FantPos  Age  G  GS  Cmp     Att     Yds  ...      TD   
277  Rk  Player  Tm  FantPos  Age  G  GS  Cmp     Att     Yds  ...      TD   
308  Rk  Player  Tm  FantPos  Age  G  GS  Cmp     Att     Yds  ...      TD   

     2PM  2PP  FantPt  PPR  DKPt  FDPt  VBD  PosRank  OvRank  
184  2PM  2PP  FantPt  PPR  DKPt  FDPt  VBD  PosRank  OvRank  
215  2PM  2PP  FantPt  PPR  DKPt  FDPt  VBD  PosRank  OvRank  
246  2PM  2PP  FantPt  PPR  DKPt  FDPt  VBD  PosRank  OvRank  
277  2PM  2PP  FantPt  PPR  DKPt  FDPt  VBD  PosRank  OvRank  
308  2PM  2PP  FantPt  PPR  DKPt  FDPt  VBD  PosRank  OvRank  

[5 rows x 33 columns]


<ipython-input-21-0738ea18cb49>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(fp[rk].head())


In [22]:
fp.drop([184,215,246,277,308], inplace=True)

In [23]:
print(fp[rk].head())

     Rk  Player  Tm  FantPos  Age  G  GS  Cmp PassAtt PassYds  ... TotalTD  \
339  Rk  Player  Tm  FantPos  Age  G  GS  Cmp     Att     Yds  ...      TD   
370  Rk  Player  Tm  FantPos  Age  G  GS  Cmp     Att     Yds  ...      TD   
401  Rk  Player  Tm  FantPos  Age  G  GS  Cmp     Att     Yds  ...      TD   
432  Rk  Player  Tm  FantPos  Age  G  GS  Cmp     Att     Yds  ...      TD   
463  Rk  Player  Tm  FantPos  Age  G  GS  Cmp     Att     Yds  ...      TD   

     2PM  2PP  FantPt  PPR  DKPt  FDPt  VBD  PosRank  OvRank  
339  2PM  2PP  FantPt  PPR  DKPt  FDPt  VBD  PosRank  OvRank  
370  2PM  2PP  FantPt  PPR  DKPt  FDPt  VBD  PosRank  OvRank  
401  2PM  2PP  FantPt  PPR  DKPt  FDPt  VBD  PosRank  OvRank  
432  2PM  2PP  FantPt  PPR  DKPt  FDPt  VBD  PosRank  OvRank  
463  2PM  2PP  FantPt  PPR  DKPt  FDPt  VBD  PosRank  OvRank  

[5 rows x 33 columns]


<ipython-input-23-0738ea18cb49>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(fp[rk].head())


In [24]:
fp.drop([339,370,401,432,463], inplace=True)

In [25]:
print(fp[rk].head())

     Rk  Player  Tm  FantPos  Age  G  GS  Cmp PassAtt PassYds  ... TotalTD  \
494  Rk  Player  Tm  FantPos  Age  G  GS  Cmp     Att     Yds  ...      TD   
525  Rk  Player  Tm  FantPos  Age  G  GS  Cmp     Att     Yds  ...      TD   
556  Rk  Player  Tm  FantPos  Age  G  GS  Cmp     Att     Yds  ...      TD   
587  Rk  Player  Tm  FantPos  Age  G  GS  Cmp     Att     Yds  ...      TD   
618  Rk  Player  Tm  FantPos  Age  G  GS  Cmp     Att     Yds  ...      TD   

     2PM  2PP  FantPt  PPR  DKPt  FDPt  VBD  PosRank  OvRank  
494  2PM  2PP  FantPt  PPR  DKPt  FDPt  VBD  PosRank  OvRank  
525  2PM  2PP  FantPt  PPR  DKPt  FDPt  VBD  PosRank  OvRank  
556  2PM  2PP  FantPt  PPR  DKPt  FDPt  VBD  PosRank  OvRank  
587  2PM  2PP  FantPt  PPR  DKPt  FDPt  VBD  PosRank  OvRank  
618  2PM  2PP  FantPt  PPR  DKPt  FDPt  VBD  PosRank  OvRank  

[5 rows x 33 columns]


<ipython-input-25-0738ea18cb49>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(fp[rk].head())


In [26]:
fp.drop([494,525,556,587,618], inplace=True)

In [27]:
print(fp[rk].head())

     Rk  Player  Tm  FantPos  Age  G  GS  Cmp PassAtt PassYds  ... TotalTD  \
649  Rk  Player  Tm  FantPos  Age  G  GS  Cmp     Att     Yds  ...      TD   

     2PM  2PP  FantPt  PPR  DKPt  FDPt  VBD  PosRank  OvRank  
649  2PM  2PP  FantPt  PPR  DKPt  FDPt  VBD  PosRank  OvRank  

[1 rows x 33 columns]


<ipython-input-27-0738ea18cb49>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(fp[rk].head())


In [28]:
fp.drop([649], inplace=True)

In [29]:
print(fp[rk].head())

Empty DataFrame
Columns: [Rk, Player, Tm, FantPos, Age, G, GS, Cmp, PassAtt, PassYds, PassTD, Int, RushAtt, RushYds, Y/A, RushTD, Tgt, Rec, RecYds, Y/R, RecTD, Fmb, FL, TotalTD, 2PM, 2PP, FantPt, PPR, DKPt, FDPt, VBD, PosRank, OvRank]
Index: []

[0 rows x 33 columns]


<ipython-input-29-0738ea18cb49>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(fp[rk].head())


In [30]:
#fp['Age'].median()
# median can't be calculated because values in Dataframe are still considered objects (not Int)

In [31]:
#for column in fp.columns:
 #   if fp[column].dtype == 'object':
  #    fp[column] = fp[column].astype('Int64')

#ValueError: invalid literal for int() with base 10: 'Christian McCaffrey*+'
# This loop was going through every column trying to convert objects to Ints
# There must be a way to single out just the columns with numbers


# Changing the data types
**Big area of opportunity**

In [32]:
# @title
fp.columns

Index(['Rk', 'Player', 'Tm', 'FantPos', 'Age', 'G', 'GS', 'Cmp', 'PassAtt',
       'PassYds', 'PassTD', 'Int', 'RushAtt', 'RushYds', 'Y/A', 'RushTD',
       'Tgt', 'Rec', 'RecYds', 'Y/R', 'RecTD', 'Fmb', 'FL', 'TotalTD', '2PM',
       '2PP', 'FantPt', 'PPR', 'DKPt', 'FDPt', 'VBD', 'PosRank', 'OvRank'],
      dtype='object')

In [33]:
fp = fp.astype({'Age':'Int64','G':'Int64','GS':'Int64','Cmp':'Int64','PassAtt':'Int64','PassYds':'Int64','PassTD':'Int64','Int':'Int64',
                'RushAtt':'Int64','RushYds':'Int64','Y/A':'Float64','RushTD':'Int64',
                'Tgt':'Int64','Rec':'Int64','RecYds':'Int64','Y/R':'Float64','RecTD':'Int64','Fmb':'Int64','FL':'Int64','TotalTD':
                'Int64','2PM':'Int64','2PP':'Int64','FantPt':'Float64','PPR':'Float64','DKPt':'Float64','FDPt':'Float64','VBD':'Float64','PosRank':'Int64','OvRank':'Int64'})

#This was all done individually by column but I'm sure there is a way to
# say all of these are Int64. I just couldn't find the way yet


In [34]:
fp.describe()

,Age,G,GS,Cmp,PassAtt,PassYds,PassTD,Int,RushAtt,RushYds,...,TotalTD,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank
count,632.0,632.0,632.0,632.0,632.0,632.0,632.0,632.0,632.0,632.0,...,632.0,58.0,28.0,528.0,543.0,543.0,542.0,75.0,632.0,78.0
mean,26.276899,11.153481,5.140823,18.674051,28.969937,203.257911,1.193038,0.68038,23.080696,96.958861,...,1.955696,1.172414,1.428571,63.145833,83.082136,86.903499,73.148708,36.373333,89.973101,39.5
std,3.126631,5.606781,5.8473,69.089538,105.949891,761.96293,4.776867,2.466716,53.654894,230.224857,...,3.051787,0.464075,0.57275,71.967811,87.349764,90.339269,80.094192,31.74296,61.07215,22.660538
min,21.0,1.0,0.0,0.0,0.0,-7.0,0.0,0.0,0.0,-13.0,...,0.0,1.0,1.0,-2.0,-2.2,-1.3,-2.2,1.0,1.0,1.0
25%,24.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,11.0,14.55,14.75,12.55,14.5,40.0,20.25
50%,26.0,13.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,1.0,34.0,47.5,50.5,39.95,29.0,79.5,39.5
75%,28.0,16.0,10.0,0.0,0.0,0.0,0.0,0.0,13.25,54.0,...,3.0,1.0,2.0,92.25,127.1,135.8,109.1,52.0,131.0,58.75
max,40.0,17.0,17.0,410.0,612.0,4624.0,36.0,21.0,280.0,1459.0,...,21.0,3.0,3.0,393.0,403.2,420.6,410.6,157.0,243.0,78.0


In [35]:
fp.Age.unique()
# This was around the time that I realized that the code needed to be run at the beginning
# of every session.

<IntegerArray>
[27, 24, 29, 25, 31, 30, 26, 22, 23, 34, 21, 33, 28, 35, 32, 38, 37, 36, 39,
 40]
Length: 20, dtype: Int64

In [36]:
age = fp['Age'].isin(['Age'])

In [37]:
print(fp[age].head())

Empty DataFrame
Columns: [Rk, Player, Tm, FantPos, Age, G, GS, Cmp, PassAtt, PassYds, PassTD, Int, RushAtt, RushYds, Y/A, RushTD, Tgt, Rec, RecYds, Y/R, RecTD, Fmb, FL, TotalTD, 2PM, 2PP, FantPt, PPR, DKPt, FDPt, VBD, PosRank, OvRank]
Index: []

[0 rows x 33 columns]


In [38]:
print(fp[rk].head())

Empty DataFrame
Columns: [Rk, Player, Tm, FantPos, Age, G, GS, Cmp, PassAtt, PassYds, PassTD, Int, RushAtt, RushYds, Y/A, RushTD, Tgt, Rec, RecYds, Y/R, RecTD, Fmb, FL, TotalTD, 2PM, 2PP, FantPt, PPR, DKPt, FDPt, VBD, PosRank, OvRank]
Index: []

[0 rows x 33 columns]


<ipython-input-38-0738ea18cb49>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(fp[rk].head())


# Getting the positions separated

In [39]:
Qb = fp['FantPos'].isin(['QB'])
Rb = fp['FantPos'].isin(['RB'])
Wr = fp['FantPos'].isin(['WR'])
Te = fp['FantPos'].isin(['TE'])

#Trying to breakdown the data into positions


## Noticed the need to find the average at each position

In [40]:
print(fp[Qb].describe())
# #This describe() pinpointed an issue of allowing in too many
#QBs that didn't play

             Age         G        GS         Cmp     PassAtt      PassYds  \
count       83.0      83.0      83.0        83.0        83.0         83.0   
mean   28.120482  8.240964  6.554217  141.963855  220.144578  1545.084337   
std     4.121323  5.507402  6.355817  137.909961  209.266438  1539.344184   
min         21.0       1.0       0.0         0.0         0.0          0.0   
25%         25.0       3.0       1.0        18.0        36.0        179.0   
50%         27.0       7.0       4.0       100.0       153.0        909.0   
75%         30.0      13.0      13.0       255.5       402.5       2699.0   
max         40.0      17.0      17.0       410.0       612.0       4624.0   

          PassTD       Int    RushAtt     RushYds  ...   TotalTD       2PM  \
count       83.0      83.0       83.0        83.0  ...      83.0       9.0   
mean         9.0  5.168675   27.53012  113.722892  ...  1.325301  1.333333   
std    10.220735  4.830844  31.561916  159.509868  ...  2.632389  0.7071

In [41]:
#ax = fp[Qb].plot(kind='density',figsize=(14,6))
#ax.set_ylabel('PPR')
#ax.set_xlabel('PassAtt')

#LinAlgError: The data appears to lie in a lower-dimensional subspace of the space
# in which it is expressed. This has resulted in a singular data covariance matrix,
#which cannot be treated using the algorithms implemented in `gaussian_kde`. Consider
#performing principle component analysis / dimensionality reduction and using `gaussian_kde` with the transformed data.

#This was just me trying to apply a lesson from FCC

In [42]:
#fp[Qb].median()

#TypeError: Cannot convert [['3' '5' '8'....]]
#not sure why this resulted in error, maybe it was too many different columns to
#pull that data

## Filtering out QBs that have played less than 5 games

In [43]:
fp[Qb].describe()
#How do I filter out QBs that have played less than 5 games?

,Age,G,GS,Cmp,PassAtt,PassYds,PassTD,Int,RushAtt,RushYds,...,TotalTD,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank
count,83.0,83.0,83.0,83.0,83.0,83.0,83.0,83.0,83.0,83.0,...,83.0,9.0,27.0,75.0,80.0,80.0,80.0,11.0,83.0,14.0
mean,28.120482,8.240964,6.554217,141.963855,220.144578,1545.084337,9.0,5.168675,27.53012,113.722892,...,1.325301,1.333333,1.444444,115.866667,108.67,118.9575,113.995,46.181818,42.0,41.428571
std,4.121323,5.507402,6.355817,137.909961,209.266438,1539.344184,10.220735,4.830844,31.561916,159.509868,...,2.632389,0.707107,0.57735,111.736821,111.856088,119.113749,115.855545,37.872993,24.103942,28.640247
min,21.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-13.0,...,0.0,1.0,1.0,-1.0,-1.3,-1.3,-1.3,7.0,1.0,3.0
25%,25.0,3.0,1.0,18.0,36.0,179.0,1.0,1.0,6.5,10.0,...,0.0,1.0,1.0,21.5,13.425,17.325,15.675,14.5,21.5,12.0
50%,27.0,7.0,4.0,100.0,153.0,909.0,4.0,4.0,17.0,54.0,...,0.0,1.0,1.0,73.0,67.15,78.15,73.4,31.0,42.0,45.0
75%,30.0,13.0,13.0,255.5,402.5,2699.0,15.0,8.5,37.5,159.0,...,1.5,1.0,2.0,213.5,197.325,218.075,206.575,70.5,62.5,64.5
max,40.0,17.0,17.0,410.0,612.0,4624.0,36.0,21.0,157.0,821.0,...,15.0,3.0,3.0,393.0,392.6,420.6,410.6,122.0,83.0,78.0


In [44]:
#AvgQb = [a for a in fp if (a['FantPos'] == 'QB' and a['G'] > 7)]

#TypeError: string indices must be integers

#Ai recommended I use iterrows()
#Qb7 = [row for _, row in fp.iterrows() if (row['FantPos'] == 'QB' and row['G'] > 7)]
#This resulted in KeyError.
#I should be iterating through the columns instead

In [45]:
#AvgQb = [column for column in fp if column['FantPos'] == 'QB' and column['G'] > 7]

#TypeError: string indices must be integers

In [46]:
fp.columns

Index(['Rk', 'Player', 'Tm', 'FantPos', 'Age', 'G', 'GS', 'Cmp', 'PassAtt',
       'PassYds', 'PassTD', 'Int', 'RushAtt', 'RushYds', 'Y/A', 'RushTD',
       'Tgt', 'Rec', 'RecYds', 'Y/R', 'RecTD', 'Fmb', 'FL', 'TotalTD', '2PM',
       '2PP', 'FantPt', 'PPR', 'DKPt', 'FDPt', 'VBD', 'PosRank', 'OvRank'],
      dtype='object')

In [47]:
#AvgQb = [column for column in fp if column[3] == 'QB' and column[5] > 7]

#IndexError: string index out of range
#This was trying check the 4th character of each column and compare it to QB
#And a column named 5 and seeing if it was greater than 7

# Quarterbacks

Maybe not the best way to find the average QB (games played)

**Although I would love to eventually find injured players and how their numbers stacked up against the rest of the league**

I imagine this could be the use of points per game/attempts/targets

In [48]:
AvgQbG = fp[(fp['FantPos'] == 'QB') & (fp['G'] > 7)]


In [49]:
AvgQbG.describe()

,Age,G,GS,Cmp,PassAtt,PassYds,PassTD,Int,RushAtt,RushYds,...,TotalTD,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank
count,38.0,38.0,38.0,38.0,38.0,38.0,38.0,38.0,38.0,38.0,...,38.0,7.0,21.0,38.0,38.0,38.0,38.0,11.0,38.0,14.0
mean,27.026316,13.473684,12.421053,267.263158,411.394737,2919.394737,17.394737,8.894737,49.342105,210.368421,...,2.526316,1.142857,1.52381,200.815789,201.015789,217.726316,209.831579,46.181818,20.605263,41.428571
std,3.872892,3.244428,4.512035,103.549357,153.153084,1210.848698,9.491218,4.260374,34.875281,189.238934,...,3.454232,0.377964,0.601585,95.978842,95.923718,100.621994,98.764919,37.872993,13.385227,28.640247
min,22.0,8.0,1.0,28.0,46.0,297.0,1.0,1.0,14.0,11.0,...,0.0,1.0,1.0,22.0,22.1,25.4,24.4,7.0,1.0,3.0
25%,24.0,10.25,9.25,204.0,314.25,2082.5,10.0,6.25,28.0,74.0,...,0.0,1.0,1.0,121.5,121.375,138.325,128.375,14.5,10.25,12.0
50%,25.5,15.0,13.0,297.0,445.5,2973.5,17.0,9.0,38.5,165.0,...,1.0,1.0,1.0,213.5,214.0,234.5,223.25,31.0,19.5,45.0
75%,28.0,16.75,16.0,361.0,545.5,4003.25,24.75,11.0,54.5,251.5,...,3.0,1.0,2.0,273.0,273.175,292.05,283.2,70.5,28.75,64.5
max,35.0,17.0,17.0,410.0,612.0,4624.0,36.0,21.0,157.0,821.0,...,15.0,2.0,3.0,393.0,392.6,420.6,410.6,122.0,56.0,78.0


## Decided to go PPR points to find Average QB

## Quarterback PPR

In [50]:
AvgQb = fp[(Qb) & (fp['PPR'] > fp['PPR'].mean())]

In [51]:
AvgQb.describe()

,Age,G,GS,Cmp,PassAtt,PassYds,PassTD,Int,RushAtt,RushYds,...,TotalTD,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank
count,36.0,36.0,36.0,36.0,36.0,36.0,36.0,36.0,36.0,36.0,...,36.0,7.0,22.0,36.0,36.0,36.0,36.0,11.0,36.0,14.0
mean,27.333333,13.472222,12.916667,278.333333,428.0,3056.75,18.555556,9.138889,51.138889,219.472222,...,2.611111,1.428571,1.5,211.805556,211.986111,229.402778,221.041667,46.181818,18.5,41.428571
std,4.105745,3.589557,4.002678,93.441502,137.625787,1083.461447,8.613758,4.155222,35.016039,190.938208,...,3.531716,0.786796,0.597614,86.824229,86.788377,90.650215,89.476594,37.872993,10.535654,28.640247
min,22.0,5.0,5.0,105.0,171.0,1101.0,5.0,3.0,9.0,2.0,...,0.0,1.0,1.0,87.0,86.8,96.3,90.3,7.0,1.0,3.0
25%,24.75,11.0,10.0,215.25,338.25,2193.5,11.75,6.75,31.75,84.5,...,0.0,1.0,1.0,138.75,139.75,152.875,146.375,14.5,9.75,12.0
50%,26.0,15.0,14.0,301.0,451.5,3102.0,19.0,9.0,39.0,180.0,...,1.5,1.0,1.0,228.0,228.75,246.75,237.5,31.0,18.5,45.0
75%,28.25,17.0,16.25,365.5,551.0,4023.0,25.25,11.25,56.75,255.5,...,3.25,1.5,2.0,274.25,274.575,292.575,284.175,70.5,27.25,64.5
max,38.0,17.0,17.0,410.0,612.0,4624.0,36.0,21.0,157.0,821.0,...,15.0,3.0,3.0,393.0,392.6,420.6,410.6,122.0,36.0,78.0


In [52]:
AvgQb.head()

,Rk,Player,Tm,FantPos,Age,G,GS,Cmp,PassAtt,PassYds,...,TotalTD,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank
2,3,Josh Allen,BUF,QB,27,17,17,385,579,4306,...,15,<NA>,3,393.0,392.6,420.6,410.6,122.0,1,3
4,5,Jalen Hurts*,PHI,QB,25,17,17,352,538,3858,...,15,<NA>,<NA>,357.0,356.8,382.8,371.8,89.0,2,5
7,8,Dak Prescott*,DAL,QB,30,17,17,410,590,4516,...,2,<NA>,2,343.0,342.8,359.8,351.8,76.0,3,8
9,10,Lamar Jackson*+,BAL,QB,26,16,16,307,457,3678,...,5,<NA>,1,331.0,331.2,350.2,338.2,65.0,4,10
17,18,Jordan Love,GNB,QB,25,17,17,372,579,4159,...,4,<NA>,2,319.0,319.1,339.1,330.1,53.0,5,18


### Average QB PPR

In [53]:
AvgQb[['Player','PassAtt','Cmp','PassYds','PassTD','Int','RushAtt','RushYds','RushTD','PPR','PosRank']].sort_values(by='PPR', ascending=False)

,Player,PassAtt,Cmp,PassYds,PassTD,Int,RushAtt,RushYds,RushTD,PPR,PosRank
2,Josh Allen,579,385,4306,29,18,111,524,15,392.6,1
4,Jalen Hurts*,538,352,3858,23,15,157,605,15,356.8,2
7,Dak Prescott*,590,410,4516,36,9,55,242,2,342.8,3
9,Lamar Jackson*+,457,307,3678,24,7,148,821,5,331.2,4
17,Jordan Love,579,372,4159,32,11,50,247,4,319.1,5
31,Brock Purdy*,444,308,4280,31,11,39,144,2,295.6,6
40,Jared Goff,605,407,4575,30,12,32,21,2,289.1,7
50,Patrick Mahomes*,597,401,4183,27,14,75,389,0,280.2,8
58,C.J. Stroud*,499,319,4108,23,5,39,167,3,276.0,9
61,Baker Mayfield*,566,364,4044,28,10,62,163,1,274.1,10


### Elite QB PPR

In [54]:
EliteQb = AvgQb[(AvgQb['FantPt'] > 274)]

In [55]:
EliteQb[['Player', 'Cmp','PassAtt','PassYds','PassTD','Int','RushAtt','RushYds','RushTD','PPR']].sort_values(by='PPR', ascending=False)

,Player,Cmp,PassAtt,PassYds,PassTD,Int,RushAtt,RushYds,RushTD,PPR
2,Josh Allen,385,579,4306,29,18,111,524,15,392.6
4,Jalen Hurts*,352,538,3858,23,15,157,605,15,356.8
7,Dak Prescott*,410,590,4516,36,9,55,242,2,342.8
9,Lamar Jackson*+,307,457,3678,24,7,148,821,5,331.2
17,Jordan Love,372,579,4159,32,11,50,247,4,319.1
31,Brock Purdy*,308,444,4280,31,11,39,144,2,295.6
40,Jared Goff,407,605,4575,30,12,32,21,2,289.1
50,Patrick Mahomes*,401,597,4183,27,14,75,389,0,280.2
58,C.J. Stroud*,319,499,4108,23,5,39,167,3,276.0


## Completions

### Attempting to break each average and elite into the different categories
**Ideally once all stat/category is calculated trends can be found**
**by comparing all of the players on every category**


### Average Completions

In [56]:
AvgQbCmp = fp[(Qb) & (fp['Cmp'] > 343)]

In [57]:
AvgQbCmp.describe()

,Age,G,GS,Cmp,PassAtt,PassYds,PassTD,Int,RushAtt,RushYds,...,TotalTD,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank
count,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,...,11.0,1.0,8.0,11.0,11.0,11.0,11.0,8.0,11.0,10.0
mean,26.909091,16.818182,16.818182,382.909091,576.181818,4191.363636,27.363636,13.272727,66.090909,264.272727,...,4.363636,1.0,1.875,298.727273,298.745455,320.927273,311.972727,50.0,8.090909,40.3
std,2.773249,0.40452,0.40452,18.327327,23.493906,278.567504,4.631905,3.87533,37.937988,190.113172,...,5.536655,<NA>,0.64087,47.839504,47.665362,49.17044,48.53671,42.213742,4.989079,29.706153
min,23.0,16.0,16.0,352.0,538.0,3858.0,21.0,8.0,32.0,21.0,...,0.0,1.0,1.0,241.0,241.1,255.1,249.1,7.0,1.0,3.0
25%,25.0,17.0,17.0,371.0,562.0,3981.0,24.0,10.5,41.5,118.5,...,0.5,1.0,1.75,266.5,266.45,290.95,281.05,15.5,4.0,10.5
50%,27.0,17.0,17.0,385.0,579.0,4159.0,28.0,14.0,55.0,247.0,...,2.0,1.0,2.0,280.0,280.2,303.2,294.2,39.0,8.0,45.0
75%,28.5,17.0,17.0,394.5,593.5,4411.0,29.5,14.5,72.5,364.0,...,4.5,1.0,2.0,331.0,330.95,349.45,340.95,79.25,11.5,64.5
max,32.0,17.0,17.0,410.0,612.0,4624.0,36.0,21.0,157.0,605.0,...,15.0,1.0,3.0,393.0,392.6,420.6,410.6,122.0,16.0,77.0


In [58]:
AvgQbCmp[['Player','PassAtt','Cmp','PassYds','PassTD','Int','RushAtt','RushYds','RushTD','PPR','PosRank']].sort_values(by='Cmp', ascending=False)

,Player,PassAtt,Cmp,PassYds,PassTD,Int,RushAtt,RushYds,RushTD,PPR,PosRank
7,Dak Prescott*,590,410,4516,36,9,55,242,2,342.8,3
40,Jared Goff,605,407,4575,30,12,32,21,2,289.1,7
50,Patrick Mahomes*,597,401,4183,27,14,75,389,0,280.2,8
67,Tua Tagovailoa*,560,388,4624,29,14,35,74,0,270.4,11
78,Sam Howell,612,388,3946,21,21,48,263,5,257.5,14
2,Josh Allen,579,385,4306,29,18,111,524,15,392.6,1
81,Derek Carr,548,375,3878,25,8,32,40,0,241.1,16
17,Jordan Love,579,372,4159,32,11,50,247,4,319.1,5
77,Trevor Lawrence,564,370,4016,21,14,70,339,4,262.5,12
61,Baker Mayfield*,566,364,4044,28,10,62,163,1,274.1,10


### Elite Completions

In [59]:
EliteQbCmp = AvgQbCmp[(AvgQbCmp['Cmp']>385)]

In [60]:
EliteQbCmp[['Player','PassAtt','Cmp','PassYds','PassTD','Int','RushAtt','RushYds','RushTD','PPR','PosRank']].sort_values(by='Cmp', ascending=False)

,Player,PassAtt,Cmp,PassYds,PassTD,Int,RushAtt,RushYds,RushTD,PPR,PosRank
7,Dak Prescott*,590,410,4516,36,9,55,242,2,342.8,3
40,Jared Goff,605,407,4575,30,12,32,21,2,289.1,7
50,Patrick Mahomes*,597,401,4183,27,14,75,389,0,280.2,8
67,Tua Tagovailoa*,560,388,4624,29,14,35,74,0,270.4,11
78,Sam Howell,612,388,3946,21,21,48,263,5,257.5,14


## Attempts

In [61]:
AvgQbAtt = fp[(Qb) & (fp['PassAtt'] > 522)]

In [62]:
AvgQbAtt.describe()

,Age,G,GS,Cmp,PassAtt,PassYds,PassTD,Int,RushAtt,RushYds,...,TotalTD,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank
count,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,...,12.0,2.0,9.0,12.0,12.0,12.0,12.0,8.0,12.0,10.0
mean,26.5,16.75,16.75,377.25,572.083333,4081.833333,26.0,13.0,63.833333,263.333333,...,4.0,1.0,1.777778,286.833333,286.883333,309.05,299.841667,50.0,9.333333,40.3
std,3.0,0.452267,0.452267,26.261361,26.520861,463.150051,6.466698,3.81385,37.008189,181.295005,...,5.427204,0.0,0.666667,61.46667,61.269581,62.376066,62.510791,42.213742,6.414945,29.706153
min,22.0,16.0,16.0,315.0,527.0,2877.0,11.0,8.0,32.0,21.0,...,0.0,1.0,1.0,156.0,156.4,178.4,166.4,7.0,1.0,3.0
25%,24.75,16.75,16.75,368.5,557.0,3929.0,22.5,10.0,38.0,140.75,...,0.0,1.0,1.0,261.5,261.25,288.75,277.625,15.5,4.5,10.5
50%,26.0,17.0,17.0,380.0,572.5,4101.5,27.5,13.0,52.5,250.0,...,2.0,1.0,2.0,277.0,277.15,298.15,289.3,39.0,9.0,45.0
75%,28.25,17.0,17.0,391.25,591.75,4358.5,29.25,14.25,71.25,351.5,...,4.25,1.0,2.0,325.0,325.025,344.275,335.525,79.25,12.5,64.5
max,32.0,17.0,17.0,410.0,612.0,4624.0,36.0,21.0,157.0,605.0,...,15.0,1.0,3.0,393.0,392.6,420.6,410.6,122.0,23.0,77.0


### Average Attempts

In [63]:
AvgQbAtt[['Player','PassAtt','Cmp','PassYds','PassTD','Int','RushAtt','RushYds','RushTD','PPR','PosRank']].sort_values(by='PassAtt', ascending=False)

,Player,PassAtt,Cmp,PassYds,PassTD,Int,RushAtt,RushYds,RushTD,PPR,PosRank
78,Sam Howell,612,388,3946,21,21,48,263,5,257.5,14
40,Jared Goff,605,407,4575,30,12,32,21,2,289.1,7
50,Patrick Mahomes*,597,401,4183,27,14,75,389,0,280.2,8
7,Dak Prescott*,590,410,4516,36,9,55,242,2,342.8,3
2,Josh Allen,579,385,4306,29,18,111,524,15,392.6,1
17,Jordan Love,579,372,4159,32,11,50,247,4,319.1,5
61,Baker Mayfield*,566,364,4044,28,10,62,163,1,274.1,10
77,Trevor Lawrence,564,370,4016,21,14,70,339,4,262.5,12
67,Tua Tagovailoa*,560,388,4624,29,14,35,74,0,270.4,11
81,Derek Carr,548,375,3878,25,8,32,40,0,241.1,16


### Elite Attempts

In [64]:
EliteQbAtt = AvgQbAtt[(AvgQbAtt['PassAtt']>591)]

In [65]:
EliteQbAtt[['Player','PassAtt','Cmp','PassYds','PassTD','Int','RushAtt','RushYds','RushTD','PPR','PosRank']].sort_values(by='PassAtt', ascending=False)

,Player,PassAtt,Cmp,PassYds,PassTD,Int,RushAtt,RushYds,RushTD,PPR,PosRank
78,Sam Howell,612,388,3946,21,21,48,263,5,257.5,14
40,Jared Goff,605,407,4575,30,12,32,21,2,289.1,7
50,Patrick Mahomes*,597,401,4183,27,14,75,389,0,280.2,8


## Pass Yards

In [66]:
AvgQbPassYds = fp[(Qb) & (fp['PassYds'] > (AvgQb['PassYds'].mean()))]

In [67]:
AvgQbPassYds.describe()

,Age,G,GS,Cmp,PassAtt,PassYds,PassTD,Int,RushAtt,RushYds,...,TotalTD,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank
count,19.0,19.0,19.0,19.0,19.0,19.0,19.0,19.0,19.0,19.0,...,19.0,3.0,14.0,19.0,19.0,19.0,19.0,11.0,19.0,14.0
mean,27.526316,16.157895,15.947368,352.315789,534.263158,3961.526316,25.473684,11.210526,61.947368,259.368421,...,3.578947,1.0,1.714286,281.210526,281.405263,301.510526,292.510526,46.181818,10.157895,41.428571
std,3.906809,1.118688,1.31122,39.925003,56.434664,446.276243,5.081212,4.02187,38.402074,208.257909,...,4.33738,0.0,0.61125,49.259606,49.001729,51.036271,50.506445,37.872993,5.899747,28.640247
min,22.0,13.0,13.0,297.0,444.0,3070.0,15.0,5.0,21.0,21.0,...,0.0,1.0,1.0,196.0,196.2,216.2,205.2,7.0,1.0,3.0
25%,25.0,15.5,15.0,313.5,494.5,3768.0,22.0,8.5,36.0,122.0,...,1.0,1.0,1.0,250.0,250.0,266.5,259.5,14.5,5.5,12.0
50%,27.0,17.0,16.0,364.0,548.0,4016.0,25.0,11.0,50.0,228.0,...,3.0,1.0,2.0,274.0,274.1,292.4,284.1,31.0,10.0,45.0
75%,29.5,17.0,17.0,386.5,579.0,4231.5,29.0,14.0,72.5,340.0,...,4.0,1.0,2.0,307.5,307.35,326.85,318.35,70.5,14.5,64.5
max,35.0,17.0,17.0,410.0,612.0,4624.0,36.0,21.0,157.0,821.0,...,15.0,1.0,3.0,393.0,392.6,420.6,410.6,122.0,21.0,78.0


### Average Pass Yards

In [68]:
AvgQbPassYds[['Player','PassAtt','Cmp','PassYds','PassTD','Int','RushAtt','RushYds','RushTD','PPR','PosRank']].sort_values(by='PassYds', ascending=False)

,Player,PassAtt,Cmp,PassYds,PassTD,Int,RushAtt,RushYds,RushTD,PPR,PosRank
67,Tua Tagovailoa*,560,388,4624,29,14,35,74,0,270.4,11
40,Jared Goff,605,407,4575,30,12,32,21,2,289.1,7
7,Dak Prescott*,590,410,4516,36,9,55,242,2,342.8,3
2,Josh Allen,579,385,4306,29,18,111,524,15,392.6,1
31,Brock Purdy*,444,308,4280,31,11,39,144,2,295.6,6
50,Patrick Mahomes*,597,401,4183,27,14,75,389,0,280.2,8
17,Jordan Love,579,372,4159,32,11,50,247,4,319.1,5
58,C.J. Stroud*,499,319,4108,23,5,39,167,3,276.0,9
61,Baker Mayfield*,566,364,4044,28,10,62,163,1,274.1,10
77,Trevor Lawrence,564,370,4016,21,14,70,339,4,262.5,12


### Elite Pass Yards

In [69]:
EliteQbPassYds = AvgQbPassYds[(AvgQbPassYds['PassYds']>4299)]

In [70]:
EliteQbPassYds[['Player','PassAtt','Cmp','PassYds','PassTD','Int','RushAtt','RushYds','RushTD','PPR','PosRank']].sort_values(by='PassYds', ascending=False)

,Player,PassAtt,Cmp,PassYds,PassTD,Int,RushAtt,RushYds,RushTD,PPR,PosRank
67,Tua Tagovailoa*,560,388,4624,29,14,35,74,0,270.4,11
40,Jared Goff,605,407,4575,30,12,32,21,2,289.1,7
7,Dak Prescott*,590,410,4516,36,9,55,242,2,342.8,3
2,Josh Allen,579,385,4306,29,18,111,524,15,392.6,1


## Touchdown Passes

In [71]:
AvgQbTDPass = fp[(Qb) & (fp['PassTD'] > AvgQb['PassTD'].mean())]

In [72]:
#AvgQbTDPass2 = fp[(Qb) & (AvgQb['PassTD'] > AvgQb['PassTD'].mean())]

#This was simply to ensure that fp and AvgQb would get the same result

In [73]:
AvgQbTDPass.describe()

,Age,G,GS,Cmp,PassAtt,PassYds,PassTD,Int,RushAtt,RushYds,...,TotalTD,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank
count,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,...,18.0,3.0,13.0,18.0,18.0,18.0,18.0,11.0,18.0,14.0
mean,27.555556,16.111111,16.111111,354.944444,536.722222,3998.0,26.055556,11.333333,63.5,268.222222,...,3.611111,1.0,1.692308,285.944444,286.138889,306.25,297.361111,46.181818,9.555556,41.428571
std,4.017934,1.131833,1.131833,39.354306,57.01373,429.086033,4.530579,4.10165,38.896923,210.583938,...,4.460795,0.0,0.630425,46.026172,45.734388,48.021077,47.197696,37.872993,5.436502,28.640247
min,22.0,13.0,13.0,297.0,444.0,3070.0,20.0,5.0,21.0,21.0,...,0.0,1.0,1.0,226.0,227.3,245.3,235.8,7.0,1.0,3.0
25%,25.0,15.25,15.25,320.0,499.0,3863.0,23.0,8.25,37.5,146.75,...,1.0,1.0,1.0,257.0,257.05,278.55,267.8,14.5,5.25,12.0
50%,26.5,16.5,16.5,367.0,554.0,4030.0,25.5,11.0,51.0,235.0,...,2.5,1.0,2.0,274.5,275.05,292.75,284.25,31.0,9.5,45.0
75%,29.75,17.0,17.0,387.25,579.0,4255.75,29.0,14.0,73.75,340.5,...,4.0,1.0,2.0,313.25,313.225,332.975,324.225,70.5,13.75,64.5
max,35.0,17.0,17.0,410.0,612.0,4624.0,36.0,21.0,157.0,821.0,...,15.0,1.0,3.0,393.0,392.6,420.6,410.6,122.0,19.0,78.0


In [74]:
#AvgQbTDPass2.describe()

#Same result as AvgQbTDPass

### Average TD Passes

In [75]:
AvgQbTDPass[['Player','PassAtt','Cmp','PassYds','PassTD','Int','RushAtt','RushYds','RushTD','PPR','PosRank']].sort_values(by='PassTD', ascending=False)


,Player,PassAtt,Cmp,PassYds,PassTD,Int,RushAtt,RushYds,RushTD,PPR,PosRank
7,Dak Prescott*,590,410,4516,36,9,55,242,2,342.8,3
17,Jordan Love,579,372,4159,32,11,50,247,4,319.1,5
31,Brock Purdy*,444,308,4280,31,11,39,144,2,295.6,6
40,Jared Goff,605,407,4575,30,12,32,21,2,289.1,7
2,Josh Allen,579,385,4306,29,18,111,524,15,392.6,1
67,Tua Tagovailoa*,560,388,4624,29,14,35,74,0,270.4,11
61,Baker Mayfield*,566,364,4044,28,10,62,163,1,274.1,10
50,Patrick Mahomes*,597,401,4183,27,14,75,389,0,280.2,8
79,Russell Wilson,447,297,3070,26,8,80,341,3,256.9,13
81,Derek Carr,548,375,3878,25,8,32,40,0,241.1,16


### Elite TD Passes

In [76]:
EliteQbTDPass = AvgQbTDPass[(AvgQbTDPass['PassTD']>30)]

In [77]:
EliteQbTDPass[['Player','PassAtt','Cmp','PassYds','PassTD','Int','RushAtt','RushYds','RushTD','PPR','PosRank']].sort_values(by='PassTD', ascending=False)

,Player,PassAtt,Cmp,PassYds,PassTD,Int,RushAtt,RushYds,RushTD,PPR,PosRank
7,Dak Prescott*,590,410,4516,36,9,55,242,2,342.8,3
17,Jordan Love,579,372,4159,32,11,50,247,4,319.1,5
31,Brock Purdy*,444,308,4280,31,11,39,144,2,295.6,6


## Interceptions

### Average Interceptions

In [78]:
AvgQbInt = fp[(Qb) & (fp['Int'] < AvgQb['Int'].mean()) & (fp['PassAtt'] > AvgQb['PassAtt'].mean())]

In [79]:
AvgQbInt.describe()

,Age,G,GS,Cmp,PassAtt,PassYds,PassTD,Int,RushAtt,RushYds,...,TotalTD,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank
count,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,...,8.0,1.0,7.0,8.0,8.0,8.0,8.0,3.0,8.0,4.0
mean,28.75,15.625,15.125,329.125,498.25,3664.125,23.625,7.75,59.625,261.75,...,2.5,1.0,1.714286,262.75,263.2125,280.2125,270.775,51.0,12.75,38.5
std,4.464143,1.407886,1.552648,41.267204,49.389849,497.601228,6.11643,1.38873,39.027234,243.861641,...,1.511858,<NA>,0.48795,51.332947,51.08943,51.184971,50.90315,34.219877,6.777062,35.038075
min,22.0,13.0,13.0,297.0,447.0,3070.0,15.0,5.0,32.0,40.0,...,0.0,1.0,1.0,196.0,196.2,216.2,205.2,12.0,3.0,8.0
25%,25.75,15.0,14.5,303.0,456.75,3262.25,20.0,7.0,36.25,141.25,...,1.75,1.0,1.5,231.25,232.475,247.475,239.475,38.5,7.75,9.5
50%,28.5,15.5,15.0,313.0,494.5,3651.0,23.5,8.0,45.5,197.5,...,3.0,1.0,2.0,249.0,249.0,265.5,257.0,65.0,14.5,34.0
75%,32.25,17.0,16.25,336.0,511.25,3935.5,25.25,9.0,61.25,266.75,...,3.0,1.0,2.0,289.0,289.8,305.8,294.925,70.5,17.5,63.0
max,35.0,17.0,17.0,410.0,590.0,4516.0,36.0,9.0,148.0,821.0,...,5.0,1.0,2.0,343.0,342.8,359.8,351.8,76.0,21.0,78.0


In [80]:
AvgQbInt[['Player','PassAtt','Cmp','PassYds','PassTD','Int','RushAtt','RushYds','RushTD','PPR','PosRank']].sort_values(by='Int', ascending=True)

,Player,PassAtt,Cmp,PassYds,PassTD,Int,RushAtt,RushYds,RushTD,PPR,PosRank
58,C.J. Stroud*,499,319,4108,23,5,39,167,3,276.0,9
9,Lamar Jackson*+,457,307,3678,24,7,148,821,5,331.2,4
82,Justin Herbert,456,297,3134,20,7,52,228,3,234.2,17
79,Russell Wilson,447,297,3070,26,8,80,341,3,256.9,13
81,Derek Carr,548,375,3878,25,8,32,40,0,241.1,16
7,Dak Prescott*,590,410,4516,36,9,55,242,2,342.8,3
84,Geno Smith*,499,323,3624,20,9,37,155,1,227.3,19
86,Gardner Minshew II*,490,305,3305,15,9,34,100,3,196.2,21


### Elite Interceptions

In [81]:
EliteQbInt = AvgQbInt[(AvgQbInt['Int']<9)]

In [82]:
EliteQbInt[['Player','PassAtt','Cmp','PassYds','PassTD','Int','RushAtt','RushYds','RushTD','PPR','PosRank']].sort_values(by='Int', ascending=True)

,Player,PassAtt,Cmp,PassYds,PassTD,Int,RushAtt,RushYds,RushTD,PPR,PosRank
58,C.J. Stroud*,499,319,4108,23,5,39,167,3,276.0,9
9,Lamar Jackson*+,457,307,3678,24,7,148,821,5,331.2,4
82,Justin Herbert,456,297,3134,20,7,52,228,3,234.2,17
79,Russell Wilson,447,297,3070,26,8,80,341,3,256.9,13
81,Derek Carr,548,375,3878,25,8,32,40,0,241.1,16


## Quarterback Rushing Attempts

In [83]:
AvgQbRush = fp[(Qb) & (fp['RushAtt'] > AvgQb['RushAtt'].mean())]

In [84]:
AvgQbRush.describe()

,Age,G,GS,Cmp,PassAtt,PassYds,PassTD,Int,RushAtt,RushYds,...,TotalTD,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank
count,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,...,12.0,1.0,9.0,12.0,12.0,12.0,12.0,6.0,12.0,8.0
mean,27.0,15.416667,15.166667,326.75,497.416667,3555.583333,22.916667,11.083333,88.666667,410.25,...,5.25,2.0,1.666667,278.166667,278.275,299.858333,289.275,63.333333,10.833333,36.25
std,3.190896,1.621354,1.898963,61.850443,83.390056,710.07291,6.999459,3.502164,37.105092,204.66386,...,4.864061,<NA>,0.707107,65.952922,65.668494,66.684419,67.297212,42.776941,7.358771,33.048665
min,24.0,13.0,12.0,227.0,370.0,2464.0,12.0,7.0,52.0,163.0,...,0.0,2.0,1.0,176.0,177.1,202.1,188.6,11.0,1.0,3.0
25%,24.75,14.5,13.0,288.25,439.5,3011.5,19.0,8.75,60.25,238.5,...,2.75,2.0,1.0,232.25,233.2,248.95,240.325,29.0,3.75,7.25
50%,26.5,16.0,16.0,329.5,497.5,3768.0,23.5,10.0,76.0,365.0,...,4.0,2.0,2.0,268.5,268.3,291.3,280.3,70.5,11.0,30.0
75%,28.0,17.0,17.0,373.75,569.25,4078.75,27.25,14.0,114.25,544.25,...,5.25,2.0,2.0,334.0,334.1,352.6,341.6,85.75,17.25,64.0
max,35.0,17.0,17.0,410.0,597.0,4516.0,36.0,18.0,157.0,821.0,...,15.0,2.0,3.0,393.0,392.6,420.6,410.6,122.0,22.0,78.0


### Average QB Rushing Attempts

In [85]:
AvgQbRush[['Player','PassAtt','Cmp','PassYds','PassTD','Int','RushAtt','RushYds','RushTD','PPR','PosRank']].sort_values(by='RushAtt', ascending=False)

,Player,PassAtt,Cmp,PassYds,PassTD,Int,RushAtt,RushYds,RushTD,PPR,PosRank
4,Jalen Hurts*,538,352,3858,23,15,157,605,15,356.8,2
9,Lamar Jackson*+,457,307,3678,24,7,148,821,5,331.2,4
83,Justin Fields,370,227,2562,16,9,124,657,4,230.2,18
2,Josh Allen,579,385,4306,29,18,111,524,15,392.6,1
79,Russell Wilson,447,297,3070,26,8,80,341,3,256.9,13
85,Joshua Dobbs,417,262,2464,13,10,77,421,6,200.7,20
50,Patrick Mahomes*,597,401,4183,27,14,75,389,0,280.2,8
77,Trevor Lawrence,564,370,4016,21,14,70,339,4,262.5,12
61,Baker Mayfield*,566,364,4044,28,10,62,163,1,274.1,10
7,Dak Prescott*,590,410,4516,36,9,55,242,2,342.8,3


Elite QB Rushing Attempts

In [86]:
EliteQbRush = AvgQbRush[(AvgQbRush['RushAtt']>130)]

In [87]:
EliteQbRush[['Player','PassAtt','Cmp','PassYds','PassTD','Int','RushAtt','RushYds','RushTD','PPR','PosRank']].sort_values(by='RushAtt', ascending=False)

,Player,PassAtt,Cmp,PassYds,PassTD,Int,RushAtt,RushYds,RushTD,PPR,PosRank
4,Jalen Hurts*,538,352,3858,23,15,157,605,15,356.8,2
9,Lamar Jackson*+,457,307,3678,24,7,148,821,5,331.2,4


## Quarterback Rush Yards

In [88]:
AvgQbRushYds = fp[(Qb) & (fp['RushYds'] > AvgQb['RushYds'].mean())]

In [89]:
AvgQbRushYds.describe()

,Age,G,GS,Cmp,PassAtt,PassYds,PassTD,Int,RushAtt,RushYds,...,TotalTD,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank
count,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,...,14.0,3.0,11.0,14.0,14.0,14.0,14.0,6.0,14.0,9.0
mean,26.285714,15.071429,15.0,325.642857,500.071429,3469.142857,22.071429,11.285714,80.714286,398.142857,...,4.928571,1.333333,1.454545,269.0,269.107143,290.321429,280.321429,70.333333,11.857143,36.111111
std,3.291681,2.555967,2.631174,69.776133,101.151421,820.313154,7.740645,4.547672,39.112124,186.274759,...,4.598734,0.57735,0.687552,73.758441,73.562203,75.856182,75.668855,35.279834,8.216841,33.680278
min,22.0,8.0,8.0,176.0,268.0,1799.0,10.0,5.0,39.0,228.0,...,0.0,1.0,1.0,146.0,146.4,157.4,151.4,17.0,1.0,3.0
25%,24.25,13.5,13.5,297.0,449.25,2925.25,17.0,8.25,50.5,248.5,...,3.0,1.0,1.0,230.75,231.2,248.45,239.575,56.0,4.25,8.0
50%,25.5,16.0,16.0,333.5,532.5,3768.0,22.0,10.0,72.5,340.0,...,4.0,1.0,1.0,260.0,260.0,288.0,277.25,70.5,12.5,18.0
75%,27.75,17.0,17.0,381.75,579.0,4123.25,26.75,14.0,103.25,498.25,...,5.0,1.5,2.0,328.0,328.175,347.425,336.175,85.75,17.75,76.0
max,35.0,17.0,17.0,410.0,612.0,4516.0,36.0,21.0,157.0,821.0,...,15.0,2.0,3.0,393.0,392.6,420.6,410.6,122.0,26.0,78.0


### Average QB Rush Yards

In [90]:
AvgQbRushYds[['Player','PassAtt','Cmp','PassYds','PassTD','Int','RushAtt','RushYds','RushTD','PPR','PosRank']].sort_values(by='RushYds', ascending=False)

,Player,PassAtt,Cmp,PassYds,PassTD,Int,RushAtt,RushYds,RushTD,PPR,PosRank
9,Lamar Jackson*+,457,307,3678,24,7,148,821,5,331.2,4
83,Justin Fields,370,227,2562,16,9,124,657,4,230.2,18
4,Jalen Hurts*,538,352,3858,23,15,157,605,15,356.8,2
2,Josh Allen,579,385,4306,29,18,111,524,15,392.6,1
85,Joshua Dobbs,417,262,2464,13,10,77,421,6,200.7,20
50,Patrick Mahomes*,597,401,4183,27,14,75,389,0,280.2,8
79,Russell Wilson,447,297,3070,26,8,80,341,3,256.9,13
77,Trevor Lawrence,564,370,4016,21,14,70,339,4,262.5,12
78,Sam Howell,612,388,3946,21,21,48,263,5,257.5,14
89,Bryce Young,527,315,2877,11,10,39,253,0,156.4,23


### Elite QB Rush Yards

In [91]:
EliteQbRushYds = AvgQbRushYds[(AvgQbRushYds['RushYds']>618)]

In [92]:
EliteQbRushYds[['Player','PassAtt','Cmp','PassYds','PassTD','Int','RushAtt','RushYds','RushTD','PPR','PosRank']].sort_values(by='RushYds', ascending=False)

,Player,PassAtt,Cmp,PassYds,PassTD,Int,RushAtt,RushYds,RushTD,PPR,PosRank
9,Lamar Jackson*+,457,307,3678,24,7,148,821,5,331.2,4
83,Justin Fields,370,227,2562,16,9,124,657,4,230.2,18


## Quarterback Rushing Touchdowns

In [93]:
AvgQbRushTD = fp[(Qb) & (fp['RushTD'] > AvgQb['RushTD'].mean())]

In [94]:
AvgQbRushTD['RushTD'].describe()

,RushTD
count,16.0
mean,5.3125
std,3.893905
min,3.0
25%,3.0
50%,4.0
75%,5.0
max,15.0


### Average QB Rushing TD

In [95]:
AvgQbRushTD[['Player','PassAtt','Cmp','PassYds','PassTD','Int','RushAtt','RushYds','RushTD','PPR','PosRank']].sort_values(by='RushTD', ascending=False)

,Player,PassAtt,Cmp,PassYds,PassTD,Int,RushAtt,RushYds,RushTD,PPR,PosRank
2,Josh Allen,579,385,4306,29,18,111,524,15,392.6,1
4,Jalen Hurts*,538,352,3858,23,15,157,605,15,356.8,2
85,Joshua Dobbs,417,262,2464,13,10,77,421,6,200.7,20
9,Lamar Jackson*+,457,307,3678,24,7,148,821,5,331.2,4
78,Sam Howell,612,388,3946,21,21,48,263,5,257.5,14
87,Desmond Ridder,388,249,2836,12,12,53,193,5,177.1,22
17,Jordan Love,579,372,4159,32,11,50,247,4,319.1,5
77,Trevor Lawrence,564,370,4016,21,14,70,339,4,262.5,12
83,Justin Fields,370,227,2562,16,9,124,657,4,230.2,18
168,Anthony Richardson,84,50,577,3,1,25,136,4,72.7,38


### Elite QB Rushing TD

In [96]:
EliteQbRushTD = AvgQbRushTD[(AvgQbRushTD['RushTD']>5)]

In [97]:
EliteQbRushTD[['Player','PassAtt','Cmp','PassYds','PassTD','Int','RushAtt','RushYds','RushTD','PPR','PosRank']].sort_values(by='RushTD', ascending=False)

,Player,PassAtt,Cmp,PassYds,PassTD,Int,RushAtt,RushYds,RushTD,PPR,PosRank
2,Josh Allen,579,385,4306,29,18,111,524,15,392.6,1
4,Jalen Hurts*,538,352,3858,23,15,157,605,15,356.8,2
85,Joshua Dobbs,417,262,2464,13,10,77,421,6,200.7,20


## Rushing Attempts (Early attempt)

In [98]:
#print(AvgQb['RushAtt']>100)
#this just printed the index of the player and T/F

In [99]:
#print(AvgQb['Player'], AvgQb['RushAtt']>100)
#if AvgQb['RushAtt']<100:
 # print(AvgQb['Player'])

#ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().


In [100]:
# AvgQb[AvgQb['RushAtt'] > 100]['Player'],AvgQb['RushAtt']

In [101]:
# AvgQb[AvgQb['RushAtt'] > 100][['Player', 'RushAtt']]


In [102]:
# EliteQb = AvgQb[(AvgQb['FantPt'] > 273)]

In [103]:
# EliteQb.describe()

In [104]:
# EliteQB = EliteQb[['Player', 'Cmp','PassAtt','PassYds','PassTD','Int','RushAtt','RushYds','RushTD','PPR']]

# Holding place

In [105]:
# print(EliteQB)

# Running Backs Attempt 1

In [106]:
fp[Rb].describe()

,Age,G,GS,Cmp,PassAtt,PassYds,PassTD,Int,RushAtt,RushYds,...,TotalTD,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank
count,149.0,149.0,149.0,149.0,149.0,149.0,149.0,149.0,149.0,149.0,...,149.0,18.0,0.0,130.0,132.0,132.0,132.0,23.0,149.0,23.0
mean,25.389262,10.852349,3.684564,0.033557,0.040268,0.275168,0.026846,0.0,78.516779,325.852349,...,2.825503,1.222222,<NA>,69.823077,87.341667,91.159848,78.038636,34.826087,78.979866,40.130435
std,2.53012,5.583973,5.572722,0.214857,0.281866,1.884715,0.199537,0.0,85.356059,367.844936,...,4.143917,0.548319,<NA>,71.976575,88.192737,90.676467,79.91805,34.812031,48.053949,23.197383
min,21.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,...,0.0,1.0,<NA>,-1.0,-0.2,-0.2,-0.2,2.0,1.0,1.0
25%,24.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,23.0,...,0.0,1.0,<NA>,11.0,14.6,14.6,11.875,10.0,38.0,18.0
50%,25.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,41.0,160.0,...,1.0,1.0,<NA>,39.0,46.45,49.6,42.2,29.0,76.0,38.0
75%,27.0,16.0,5.0,0.0,0.0,0.0,0.0,0.0,142.0,613.0,...,4.0,1.0,<NA>,123.0,154.45,160.45,133.175,53.5,117.0,62.0
max,33.0,17.0,17.0,2.0,3.0,17.0,2.0,0.0,280.0,1459.0,...,21.0,3.0,<NA>,324.0,391.3,399.3,357.8,157.0,166.0,73.0


In [107]:
AvgRb = fp[(fp['FantPos'] == 'RB') & (fp['G'] >10)]

In [108]:
AvgRb.head()

,Rk,Player,Tm,FantPos,Age,G,GS,Cmp,PassAtt,PassYds,...,TotalTD,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank
0,1,Christian McCaffrey*+,SFO,RB,27,16,16,0,0,0,...,21,<NA>,<NA>,324.0,391.3,399.3,357.8,157.0,1,1
5,6,Raheem Mostert*,MIA,RB,31,15,15,0,0,0,...,21,<NA>,<NA>,243.0,267.7,274.7,255.2,80.0,2,6
11,12,Travis Etienne,JAX,RB,24,17,17,0,0,0,...,12,2,<NA>,224.0,282.4,288.4,253.4,62.0,3,12
12,13,Kyren Williams*,LAR,RB,23,12,11,0,0,0,...,15,1,<NA>,223.0,255.0,263.0,239.0,61.0,4,13
15,16,Derrick Henry*,TEN,RB,29,17,17,2,3,14,...,12,<NA>,<NA>,219.0,246.7,252.7,232.7,57.0,5,16


In [109]:
AvgRb.describe()

,Age,G,GS,Cmp,PassAtt,PassYds,PassTD,Int,RushAtt,RushYds,...,TotalTD,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank
count,87.0,87.0,87.0,87.0,87.0,87.0,87.0,87.0,87.0,87.0,...,87.0,18.0,0.0,84.0,84.0,84.0,84.0,23.0,87.0,23.0
mean,25.574713,15.091954,5.954023,0.057471,0.068966,0.471264,0.045977,0.0,122.195402,510.701149,...,4.494253,1.222222,<NA>,99.309524,125.992857,131.397619,112.62381,34.826087,51.482759,40.130435
std,2.652915,1.980323,6.278253,0.279381,0.367047,2.45347,0.260056,0.0,85.818472,373.485628,...,4.647764,0.548319,<NA>,72.419124,87.654586,89.417366,79.831208,34.812031,36.875927,23.197383
min,21.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,<NA>,2.0,5.2,5.2,4.2,2.0,1.0,1.0
25%,24.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,49.5,180.0,...,1.0,1.0,<NA>,37.5,45.825,48.825,42.475,10.0,22.5,18.0
50%,25.0,16.0,3.0,0.0,0.0,0.0,0.0,0.0,104.0,416.0,...,3.0,1.0,<NA>,79.5,110.9,117.9,95.8,29.0,46.0,38.0
75%,27.5,17.0,12.5,0.0,0.0,0.0,0.0,0.0,204.5,811.5,...,6.5,1.0,<NA>,162.5,196.825,204.65,180.325,53.5,70.5,62.0
max,33.0,17.0,17.0,2.0,3.0,17.0,2.0,0.0,280.0,1459.0,...,21.0,3.0,<NA>,324.0,391.3,399.3,357.8,157.0,163.0,73.0


In [110]:
AvgRb[['Tgt','Rec','RecYds','RecTD']].describe()

,Tgt,Rec,RecYds,RecTD
count,87.0,87.0,87.0,87.0
mean,33.275862,25.816092,184.551724,0.977011
std,24.38866,19.439548,149.489967,1.478348
min,0.0,0.0,0.0,0.0
25%,12.0,9.5,50.5,0.0
50%,31.0,24.0,170.0,0.0
75%,49.0,38.5,241.0,1.5
max,95.0,76.0,591.0,7.0


In [111]:
AvgRbRec = AvgRb[(AvgRb['Rec']>25)]

In [112]:
AvgRbRec[['Tgt','Rec','RecYds','RecTD','PPR']].describe()

,Tgt,Rec,RecYds,RecTD,PPR
count,41.0,41.0,41.0,41.0,41.0
mean,54.682927,42.707317,304.658537,1.707317,191.978049
std,16.691973,13.822525,123.619297,1.706515,62.975442
min,33.0,27.0,154.0,0.0,87.6
25%,40.0,30.0,206.0,0.0,145.7
50%,51.0,39.0,259.0,1.0,195.5
75%,67.0,52.0,385.0,3.0,232.7
max,95.0,76.0,591.0,7.0,391.3


In [113]:
EliteRbRec = AvgRbRec[(AvgRbRec['Rec']>52)]

In [114]:
EliteRbRec[['Player','Tgt','Rec','RecYds','RecTD','PPR']].sort_values(by='Rec', ascending=False)

,Player,Tgt,Rec,RecYds,RecTD,PPR
18,Breece Hall,95,76,591,4,290.5
88,Alvin Kamara,86,75,466,1,233.0
0,Christian McCaffrey*+,83,67,564,7,391.3
23,Rachaad White,70,64,549,3,267.9
100,Jaylen Warren,74,61,370,0,196.4
11,Travis Etienne,73,58,476,1,282.4
38,Bijan Robinson,86,58,487,4,246.3
64,Tony Pollard,67,55,311,0,222.6


In [115]:
AvgRb[AvgRb['RushAtt'] > 122][['Player', 'RushAtt','RushTD','RecTD','FantPt','Tgt','PPR']].describe()
#AvgQb[AvgQb['RushAtt'] > 100][['Player', 'RushAtt']]


,RushAtt,RushTD,RecTD,FantPt,Tgt,PPR
count,40.0,40.0,40.0,40.0,40.0,40.0
mean,205.425,6.275,1.65,160.8,50.425,199.695
std,41.468485,4.031844,1.747526,51.389014,20.592365,61.004443
min,129.0,0.0,0.0,61.0,13.0,82.7
25%,179.75,4.0,0.0,130.0,36.75,169.025
50%,208.5,5.0,1.0,164.5,48.5,198.7
75%,234.0,8.25,3.0,189.25,65.5,235.275
max,280.0,18.0,7.0,324.0,95.0,391.3


In [116]:
EliteRbRushAtt = AvgRb[(AvgRb['RushAtt']>122)]

In [117]:
AvgRbTgt = AvgRb[(AvgRb['Tgt']>50)]

In [118]:
AvgRbTgt[['Tgt','FantPt','PPR']].describe()

,Tgt,FantPt,PPR
count,21.0,21.0,21.0
mean,68.238095,164.428571,217.857143
std,11.907581,57.393006,63.024246
min,51.0,71.0,121.3
25%,59.0,132.0,179.2
50%,67.0,167.0,222.6
75%,74.0,190.0,246.3
max,95.0,324.0,391.3


In [119]:
EliteRbTgt = AvgRbTgt[(AvgRbTgt['PPR']>246)]

In [120]:
EliteRbTgt[['Player','Tgt','Rec','RecYds','RecTD','PPR']].sort_values(by='PPR', ascending=False)

,Player,Tgt,Rec,RecYds,RecTD,PPR
0,Christian McCaffrey*+,83,67,564,7,391.3
18,Breece Hall,95,76,591,4,290.5
11,Travis Etienne,73,58,476,1,282.4
23,Rachaad White,70,64,549,3,267.9
16,Joe Mixon,64,52,376,3,267.0
38,Bijan Robinson,86,58,487,4,246.3


In [121]:
EliteRb = AvgRb[(AvgRb['FantPt'] > 189)]

In [122]:
EliteRb.describe()

,Age,G,GS,Cmp,PassAtt,PassYds,PassTD,Int,RushAtt,RushYds,...,TotalTD,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank
count,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,...,10.0,5.0,0.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0
mean,25.4,15.7,14.3,0.2,0.3,1.4,0.2,0.0,240.9,1076.8,...,13.5,1.2,<NA>,224.8,271.78,278.68,248.28,62.8,5.5,17.5
std,3.169297,1.702939,4.398232,0.632456,0.948683,4.427189,0.632456,0.0,33.093974,150.950176,...,4.326918,0.447214,<NA>,38.279092,48.033387,48.270667,42.250649,36.300597,3.02765,10.637982
min,21.0,12.0,3.0,0.0,0.0,0.0,0.0,0.0,182.0,945.0,...,9.0,1.0,<NA>,190.0,207.2,214.2,199.2,30.0,1.0,1.0
25%,23.25,15.0,14.25,0.0,0.0,0.0,0.0,0.0,220.0,997.5,...,11.25,1.0,<NA>,206.75,248.775,255.275,233.5,45.5,3.25,12.25
50%,25.0,16.5,16.0,0.0,0.0,0.0,0.0,0.0,242.5,1013.5,...,12.0,1.0,<NA>,217.0,267.35,273.85,240.0,55.5,5.5,16.5
75%,27.0,17.0,17.0,0.0,0.0,0.0,0.0,0.0,270.75,1116.5,...,14.5,1.0,<NA>,223.75,278.775,285.275,253.175,61.75,7.75,22.75
max,31.0,17.0,17.0,2.0,3.0,14.0,2.0,0.0,280.0,1459.0,...,21.0,2.0,<NA>,324.0,391.3,399.3,357.8,157.0,10.0,35.0


In [123]:
EliteRb[['Player','RushAtt','RushYds','RushTD','Tgt','Rec','RecTD','PPR']]

,Player,RushAtt,RushYds,RushTD,Tgt,Rec,RecTD,PPR
0,Christian McCaffrey*+,272,1459,14,83,67,7,391.3
5,Raheem Mostert*,209,1012,18,32,25,3,267.7
11,Travis Etienne,267,1008,11,73,58,1,282.4
12,Kyren Williams*,228,1144,12,48,32,3,255.0
15,Derrick Henry*,280,1167,12,36,28,0,246.7
16,Joe Mixon,257,1034,9,64,52,3,267.0
18,Breece Hall,223,994,5,95,76,4,290.5
23,Rachaad White,272,990,6,70,64,3,267.9
32,David Montgomery,219,1015,13,24,16,0,207.2
35,Jahmyr Gibbs*,182,945,10,71,52,1,242.1


In [124]:
# prompt: how to order by PPR EliteRb[['Player','RushAtt','RushYds','RushTD','Tgt','Rec','RecTD','PPR']]

EliteRb[['Player','RushAtt','RushYds','RushTD','Tgt','Rec','RecTD','PPR']].sort_values(by='PPR', ascending=False)


,Player,RushAtt,RushYds,RushTD,Tgt,Rec,RecTD,PPR
0,Christian McCaffrey*+,272,1459,14,83,67,7,391.3
18,Breece Hall,223,994,5,95,76,4,290.5
11,Travis Etienne,267,1008,11,73,58,1,282.4
23,Rachaad White,272,990,6,70,64,3,267.9
5,Raheem Mostert*,209,1012,18,32,25,3,267.7
16,Joe Mixon,257,1034,9,64,52,3,267.0
12,Kyren Williams*,228,1144,12,48,32,3,255.0
15,Derrick Henry*,280,1167,12,36,28,0,246.7
35,Jahmyr Gibbs*,182,945,10,71,52,1,242.1
32,David Montgomery,219,1015,13,24,16,0,207.2


In [125]:
EliteRb[['Player','RushAtt','RushYds','RushTD','Tgt','Rec','RecTD','RecYds','PPR']].sort_values(by='PPR', ascending=False)


,Player,RushAtt,RushYds,RushTD,Tgt,Rec,RecTD,RecYds,PPR
0,Christian McCaffrey*+,272,1459,14,83,67,7,564,391.3
18,Breece Hall,223,994,5,95,76,4,591,290.5
11,Travis Etienne,267,1008,11,73,58,1,476,282.4
23,Rachaad White,272,990,6,70,64,3,549,267.9
5,Raheem Mostert*,209,1012,18,32,25,3,175,267.7
16,Joe Mixon,257,1034,9,64,52,3,376,267.0
12,Kyren Williams*,228,1144,12,48,32,3,206,255.0
15,Derrick Henry*,280,1167,12,36,28,0,214,246.7
35,Jahmyr Gibbs*,182,945,10,71,52,1,316,242.1
32,David Montgomery,219,1015,13,24,16,0,117,207.2


In [126]:
fp.columns

Index(['Rk', 'Player', 'Tm', 'FantPos', 'Age', 'G', 'GS', 'Cmp', 'PassAtt',
       'PassYds', 'PassTD', 'Int', 'RushAtt', 'RushYds', 'Y/A', 'RushTD',
       'Tgt', 'Rec', 'RecYds', 'Y/R', 'RecTD', 'Fmb', 'FL', 'TotalTD', '2PM',
       '2PP', 'FantPt', 'PPR', 'DKPt', 'FDPt', 'VBD', 'PosRank', 'OvRank'],
      dtype='object')

# Running Backs (Final)

In [127]:
AvgRb = fp[(fp['PPR'] > fp['PPR'].mean()) & (Rb)]

#AvgQb = fp[(Qb) & (fp['PPR'] > fp['PPR'].mean())]

In [128]:
RbStats = fp[['Player','RushAtt','RushYds','Y/A','RushTD','Tgt','Rec','RecYds','Y/R','RecTD','TotalTD','PPR','PosRank']]

In [129]:
AvgRb.columns

Index(['Rk', 'Player', 'Tm', 'FantPos', 'Age', 'G', 'GS', 'Cmp', 'PassAtt',
       'PassYds', 'PassTD', 'Int', 'RushAtt', 'RushYds', 'Y/A', 'RushTD',
       'Tgt', 'Rec', 'RecYds', 'Y/R', 'RecTD', 'Fmb', 'FL', 'TotalTD', '2PM',
       '2PP', 'FantPt', 'PPR', 'DKPt', 'FDPt', 'VBD', 'PosRank', 'OvRank'],
      dtype='object')

In [130]:
AvgRb[['Player','RushAtt','RushYds','Y/A','RushTD','Tgt','Rec','RecYds','Y/R','RecTD','TotalTD','PPR','PosRank']].describe()

,RushAtt,RushYds,Y/A,RushTD,Tgt,Rec,RecYds,Y/R,RecTD,TotalTD,PPR,PosRank
count,53.0,53.0,53.0,53.0,53.0,53.0,53.0,53.0,53.0,53.0,53.0,53.0
mean,175.773585,746.754717,4.271698,5.415094,47.773585,37.207547,270.226415,7.367547,1.566038,6.981132,180.069811,27.150943
std,65.262087,287.686589,0.745372,3.968307,19.70127,16.01965,127.663868,1.727966,1.635066,4.491404,64.849466,15.702029
min,21.0,60.0,2.86,0.0,13.0,11.0,77.0,4.85,0.0,1.0,84.4,1.0
25%,129.0,462.0,3.89,2.0,36.0,27.0,183.0,6.14,0.0,4.0,121.3,14.0
50%,183.0,784.0,4.16,5.0,43.0,33.0,223.0,7.23,1.0,6.0,182.5,27.0
75%,223.0,990.0,4.58,7.0,63.0,50.0,368.0,8.1,3.0,9.0,222.6,40.0
max,280.0,1459.0,7.77,18.0,95.0,76.0,591.0,15.0,7.0,21.0,391.3,56.0


## Running Backs PPR

### Average Running Back PPR

In [131]:
AvgRb[['Player','RushAtt','RushYds','Y/A','RushTD','Tgt','Rec','RecYds','Y/R','RecTD','PPR','PosRank']].sort_values(by='PPR', ascending=False)

,Player,RushAtt,RushYds,Y/A,RushTD,Tgt,Rec,RecYds,Y/R,RecTD,PPR,PosRank
0,Christian McCaffrey*+,272,1459,5.36,14,83,67,564,8.42,7,391.3,1
18,Breece Hall,223,994,4.46,5,95,76,591,7.78,4,290.5,7
11,Travis Etienne,267,1008,3.78,11,73,58,476,8.21,1,282.4,3
23,Rachaad White,272,990,3.64,6,70,64,549,8.58,3,267.9,8
5,Raheem Mostert*,209,1012,4.84,18,32,25,175,7.0,3,267.7,2
16,Joe Mixon,257,1034,4.02,9,64,52,376,7.23,3,267.0,6
12,Kyren Williams*,228,1144,5.02,12,48,32,206,6.44,3,255.0,4
15,Derrick Henry*,280,1167,4.17,12,36,28,214,7.64,0,246.7,5
38,Bijan Robinson,214,976,4.56,4,86,58,487,8.4,4,246.3,12
35,Jahmyr Gibbs*,182,945,5.19,10,71,52,316,6.08,1,242.1,10


### Elite Running Back PPR

In [132]:
EliteRb = AvgRb[(AvgRb['PPR']>222)]

In [133]:
EliteRb[['Player','RushAtt','RushYds','Y/A','RushTD','Tgt','Rec','RecYds','Y/R','RecTD','PPR','PosRank']].sort_values(by='PPR', ascending=False)

,Player,RushAtt,RushYds,Y/A,RushTD,Tgt,Rec,RecYds,Y/R,RecTD,PPR,PosRank
0,Christian McCaffrey*+,272,1459,5.36,14,83,67,564,8.42,7,391.3,1
18,Breece Hall,223,994,4.46,5,95,76,591,7.78,4,290.5,7
11,Travis Etienne,267,1008,3.78,11,73,58,476,8.21,1,282.4,3
23,Rachaad White,272,990,3.64,6,70,64,549,8.58,3,267.9,8
5,Raheem Mostert*,209,1012,4.84,18,32,25,175,7.0,3,267.7,2
16,Joe Mixon,257,1034,4.02,9,64,52,376,7.23,3,267.0,6
12,Kyren Williams*,228,1144,5.02,12,48,32,206,6.44,3,255.0,4
15,Derrick Henry*,280,1167,4.17,12,36,28,214,7.64,0,246.7,5
38,Bijan Robinson,214,976,4.56,4,86,58,487,8.4,4,246.3,12
35,Jahmyr Gibbs*,182,945,5.19,10,71,52,316,6.08,1,242.1,10


## Hopefully cleaner code that will save space

In [134]:
AvgRbRushAtt = AvgRb[(AvgRb['RushAtt'] > AvgRb['RushAtt'].mean())]
AvgRbRushYds = AvgRb[(AvgRb['RushYds'] > AvgRb['RushYds'].mean())]
AvgRbYdsPerAtt = AvgRb[(AvgRb['Y/A'] > AvgRb['Y/A'].mean())]
AvgRbRushTD = AvgRb[(AvgRb['RushTD'] > AvgRb['RushTD'].mean())]
AvgRbTgt = AvgRb[(AvgRb['Tgt'] > AvgRb['Tgt'].mean())]
AvgRbRec = AvgRb[(AvgRb['Rec'] > AvgRb['Rec'].mean())]
AvgRbRecYds = AvgRb[(AvgRb['RecYds'] > AvgRb['RecYds'].mean())]
AvgRbYdsPerRec = AvgRb[(AvgRb['Y/R'] > AvgRb['Y/R'].mean())]
AvgRbRecTD = AvgRb[(AvgRb['RecTD'] > AvgRb['RecTD'].mean())]
AvgRbTotalTD = AvgRb[(AvgRb['TotalTD'] > AvgRb['TotalTD'].mean())]

## Rush Attempts

In [135]:
AvgRbRushAtt[RbStats.columns].describe()

,RushAtt,RushYds,Y/A,RushTD,Tgt,Rec,RecYds,Y/R,RecTD,TotalTD,PPR,PosRank
count,33.0,33.0,33.0,33.0,33.0,33.0,33.0,33.0,33.0,33.0,33.0,33.0
mean,219.060606,917.757576,4.181212,6.878788,52.69697,41.060606,302.757576,7.594242,1.939394,8.818182,215.439394,18.0
std,31.343799,182.910502,0.528049,4.036997,20.392224,16.486622,131.075367,2.012531,1.784296,4.40493,52.851939,11.056672
min,178.0,613.0,3.44,0.0,13.0,12.0,117.0,4.85,0.0,2.0,117.6,1.0
25%,186.0,794.0,3.79,4.0,37.0,29.0,193.0,6.14,0.0,6.0,182.5,9.0
50%,217.0,945.0,4.09,6.0,49.0,39.0,259.0,7.23,2.0,9.0,207.2,17.0
75%,238.0,1015.0,4.56,9.0,67.0,52.0,376.0,8.42,3.0,11.0,246.3,26.0
max,280.0,1459.0,5.36,18.0,95.0,76.0,591.0,15.0,7.0,21.0,391.3,40.0


In [136]:
# @title
# Use a list of numbers between 0 and 1 for the percentiles argument
## This is not what I wanted to do here but it might help in the future
AvgRbRushAtt.describe(percentiles=[0.25,0.5,0.75])

,Age,G,GS,Cmp,PassAtt,PassYds,PassTD,Int,RushAtt,RushYds,...,TotalTD,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank
count,33.0,33.0,33.0,33.0,33.0,33.0,33.0,33.0,33.0,33.0,...,33.0,14.0,0.0,33.0,33.0,33.0,33.0,22.0,33.0,22.0
mean,25.212121,15.606061,12.757576,0.090909,0.121212,0.606061,0.090909,0.0,219.060606,917.757576,...,8.818182,1.285714,<NA>,174.424242,215.439394,222.469697,194.909091,36.181818,18.0,38.818182
std,2.328594,1.560036,3.992417,0.384353,0.545297,2.6213,0.384353,0.0,31.343799,182.910502,...,4.40493,0.61125,<NA>,44.816313,52.851939,52.99551,48.322008,35.004267,11.056672,22.852813
min,21.0,12.0,3.0,0.0,0.0,0.0,0.0,0.0,178.0,613.0,...,2.0,1.0,<NA>,96.0,117.6,123.6,106.6,2.0,1.0,1.0
25%,24.0,14.0,12.0,0.0,0.0,0.0,0.0,0.0,186.0,794.0,...,6.0,1.0,<NA>,144.0,182.5,188.5,162.6,11.25,9.0,17.5
50%,25.0,16.0,13.0,0.0,0.0,0.0,0.0,0.0,217.0,945.0,...,9.0,1.0,<NA>,170.0,207.2,214.2,189.2,29.0,17.0,37.5
75%,27.0,17.0,16.0,0.0,0.0,0.0,0.0,0.0,238.0,1015.0,...,11.0,1.0,<NA>,191.0,246.3,252.7,217.3,53.75,26.0,60.5
max,31.0,17.0,17.0,2.0,3.0,14.0,2.0,0.0,280.0,1459.0,...,21.0,3.0,<NA>,324.0,391.3,399.3,357.8,157.0,40.0,73.0


### Average Rush Attempts

In [137]:
AvgRbRushAtt[RbStats.columns].sort_values(by='RushAtt', ascending=False)

,Player,RushAtt,RushYds,Y/A,RushTD,Tgt,Rec,RecYds,Y/R,RecTD,TotalTD,PPR,PosRank
15,Derrick Henry*,280,1167,4.17,12,36,28,214,7.64,0,12,246.7,5
0,Christian McCaffrey*+,272,1459,5.36,14,83,67,564,8.42,7,21,391.3,1
23,Rachaad White,272,990,3.64,6,70,64,549,8.58,3,9,267.9,8
11,Travis Etienne,267,1008,3.78,11,73,58,476,8.21,1,12,282.4,3
16,Joe Mixon,257,1034,4.02,9,64,52,376,7.23,3,12,267.0,6
66,Najee Harris,255,1035,4.06,8,38,29,170,5.86,0,8,195.5,20
64,Tony Pollard,252,1005,3.99,6,67,55,311,5.65,0,6,222.6,18
43,Saquon Barkley,247,962,3.89,6,60,41,280,6.83,4,10,223.2,13
94,Chuba Hubbard,238,902,3.79,5,44,39,233,5.97,0,5,182.5,26
37,James Cook*,237,1122,4.73,2,54,44,445,10.11,4,6,232.7,11


### Elite Rush Attempts

In [138]:
EliteRbRushAtt = AvgRbRushAtt[(AvgRbRushAtt['RushAtt']>238)]

In [139]:
EliteRbRushAtt[RbStats.columns].sort_values(by='RushAtt', ascending = False)

,Player,RushAtt,RushYds,Y/A,RushTD,Tgt,Rec,RecYds,Y/R,RecTD,TotalTD,PPR,PosRank
15,Derrick Henry*,280,1167,4.17,12,36,28,214,7.64,0,12,246.7,5
0,Christian McCaffrey*+,272,1459,5.36,14,83,67,564,8.42,7,21,391.3,1
23,Rachaad White,272,990,3.64,6,70,64,549,8.58,3,9,267.9,8
11,Travis Etienne,267,1008,3.78,11,73,58,476,8.21,1,12,282.4,3
16,Joe Mixon,257,1034,4.02,9,64,52,376,7.23,3,12,267.0,6
66,Najee Harris,255,1035,4.06,8,38,29,170,5.86,0,8,195.5,20
64,Tony Pollard,252,1005,3.99,6,67,55,311,5.65,0,6,222.6,18
43,Saquon Barkley,247,962,3.89,6,60,41,280,6.83,4,10,223.2,13


## Rush Yards

In [140]:
AvgRbRushYds[RbStats.columns].describe()

,RushAtt,RushYds,Y/A,RushTD,Tgt,Rec,RecYds,Y/R,RecTD,TotalTD,PPR,PosRank
count,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0
mean,222.0,970.607143,4.465357,7.678571,53.107143,41.428571,298.821429,7.342857,1.964286,9.642857,225.614286,15.107143
std,38.114836,147.138217,0.833878,3.887812,19.838868,15.982464,134.111356,1.931541,1.875419,4.32233,47.873272,9.117908
min,103.0,774.0,3.45,2.0,13.0,12.0,117.0,4.85,0.0,4.0,167.3,1.0
25%,207.25,876.75,3.99,5.0,37.0,28.75,196.0,6.0775,0.0,6.0,194.3,7.75
50%,221.0,983.0,4.255,6.5,51.5,40.0,251.5,7.17,2.0,9.0,212.55,14.5
75%,248.25,1034.25,4.7575,10.25,67.75,52.75,371.5,8.0525,3.0,12.0,248.775,21.5
max,280.0,1459.0,7.77,18.0,95.0,76.0,591.0,15.0,7.0,21.0,391.3,32.0


### Average Rush Yards

In [141]:
AvgRbRushYds[RbStats.columns].sort_values(by='RushYds', ascending=False)

,Player,RushAtt,RushYds,Y/A,RushTD,Tgt,Rec,RecYds,Y/R,RecTD,TotalTD,PPR,PosRank
0,Christian McCaffrey*+,272,1459,5.36,14,83,67,564,8.42,7,21,391.3,1
15,Derrick Henry*,280,1167,4.17,12,36,28,214,7.64,0,12,246.7,5
12,Kyren Williams*,228,1144,5.02,12,48,32,206,6.44,3,15,255.0,4
37,James Cook*,237,1122,4.73,2,54,44,445,10.11,4,6,232.7,11
74,D'Andre Swift*,229,1049,4.58,5,49,39,214,5.49,1,6,199.3,23
52,James Conner,208,1040,5.0,7,33,27,165,6.11,2,9,201.5,15
66,Najee Harris,255,1035,4.06,8,38,29,170,5.86,0,8,195.5,20
16,Joe Mixon,257,1034,4.02,9,64,52,376,7.23,3,12,267.0,6
32,David Montgomery,219,1015,4.63,13,24,16,117,7.31,0,13,207.2,9
5,Raheem Mostert*,209,1012,4.84,18,32,25,175,7.0,3,21,267.7,2


### Elite Rush Yards


In [142]:
EliteRbRushYds = AvgRbRushYds[(AvgRbRushYds['RushYds']>1034)]

In [143]:
EliteRbRushYds[RbStats.columns].sort_values(by='RushYds', ascending=False)

,Player,RushAtt,RushYds,Y/A,RushTD,Tgt,Rec,RecYds,Y/R,RecTD,TotalTD,PPR,PosRank
0,Christian McCaffrey*+,272,1459,5.36,14,83,67,564,8.42,7,21,391.3,1
15,Derrick Henry*,280,1167,4.17,12,36,28,214,7.64,0,12,246.7,5
12,Kyren Williams*,228,1144,5.02,12,48,32,206,6.44,3,15,255.0,4
37,James Cook*,237,1122,4.73,2,54,44,445,10.11,4,6,232.7,11
74,D'Andre Swift*,229,1049,4.58,5,49,39,214,5.49,1,6,199.3,23
52,James Conner,208,1040,5.0,7,33,27,165,6.11,2,9,201.5,15
66,Najee Harris,255,1035,4.06,8,38,29,170,5.86,0,8,195.5,20


## Yards Per Attempt

In [144]:
AvgRbYdsPerAtt[RbStats.columns].describe()

,RushAtt,RushYds,Y/A,RushTD,Tgt,Rec,RecYds,Y/R,RecTD,TotalTD,PPR,PosRank
count,25.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0
mean,159.68,768.16,4.8172,5.6,48.48,37.72,270.76,7.0744,1.72,7.32,184.2,26.6
std,63.406572,315.482789,0.697272,4.618802,20.314445,16.027372,139.434058,1.184856,1.744515,5.617829,73.690015,17.136705
min,53.0,238.0,4.28,1.0,23.0,16.0,117.0,5.16,0.0,1.0,97.4,1.0
25%,102.0,461.0,4.49,2.0,36.0,27.0,175.0,6.11,0.0,3.0,112.5,11.0
50%,169.0,794.0,4.61,4.0,40.0,32.0,209.0,7.11,1.0,6.0,190.7,27.0
75%,214.0,1012.0,5.0,7.0,56.0,50.0,370.0,7.77,3.0,9.0,232.7,42.0
max,272.0,1459.0,7.77,18.0,95.0,76.0,591.0,10.11,7.0,21.0,391.3,52.0


### Average Yards/Attempt

In [145]:
AvgRbYdsPerAtt[RbStats.columns].sort_values(by='Y/A', ascending=False)

,Player,RushAtt,RushYds,Y/A,RushTD,Tgt,Rec,RecYds,Y/R,RecTD,TotalTD,PPR,PosRank
70,De'Von Achane,103,800,7.77,8,37,27,197,7.3,3,11,190.7,21
161,Jaleel McLaughlin,76,410,5.39,1,36,31,160,5.16,2,3,106.0,46
0,Christian McCaffrey*+,272,1459,5.36,14,83,67,564,8.42,7,21,391.3,1
100,Jaylen Warren,149,784,5.26,4,74,61,370,6.07,0,4,196.4,30
35,Jahmyr Gibbs*,182,945,5.19,10,71,52,316,6.08,1,11,242.1,10
12,Kyren Williams*,228,1144,5.02,12,48,32,206,6.44,3,15,255.0,4
52,James Conner,208,1040,5.0,7,33,27,165,6.11,2,9,201.5,15
5,Raheem Mostert*,209,1012,4.84,18,32,25,175,7.0,3,21,267.7,2
37,James Cook*,237,1122,4.73,2,54,44,445,10.11,4,6,232.7,11
32,David Montgomery,219,1015,4.63,13,24,16,117,7.31,0,13,207.2,9


### Elite Yards/Attempt

In [146]:
EliteRbYdsPerAtt = AvgRbYdsPerAtt[(AvgRbYdsPerAtt['Y/A']>7)]

In [147]:
EliteRbYdsPerAtt[RbStats.columns].sort_values(by='Y/A', ascending=False)

,Player,RushAtt,RushYds,Y/A,RushTD,Tgt,Rec,RecYds,Y/R,RecTD,TotalTD,PPR,PosRank
70,De'Von Achane,103,800,7.77,8,37,27,197,7.3,3,11,190.7,21


## Rushing TD

In [148]:
AvgRbRushTD[RbStats.columns].describe()

,RushAtt,RushYds,Y/A,RushTD,Tgt,Rec,RecYds,Y/R,RecTD,TotalTD,PPR,PosRank
count,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0
mean,220.5,961.15,4.4505,9.5,46.65,36.45,266.6,7.4895,1.7,11.2,225.855,13.6
std,46.863183,202.55013,0.98656,3.300718,20.191908,16.788075,134.847438,2.062526,1.838191,4.073018,56.730967,9.500139
min,103.0,411.0,3.04,6.0,13.0,12.0,84.0,5.55,0.0,6.0,118.5,1.0
25%,203.25,881.25,3.965,7.0,32.75,26.5,173.75,6.1025,0.0,9.0,194.3,5.75
50%,223.5,997.5,4.15,8.5,43.0,30.5,229.0,7.265,1.0,10.5,218.25,13.5
75%,255.5,1034.25,4.88,12.0,64.75,52.0,312.25,8.09,3.0,12.25,258.0,18.5
max,280.0,1459.0,7.77,18.0,83.0,67.0,564.0,15.0,7.0,21.0,391.3,37.0


### Average Rushing TD

In [149]:
AvgRbRushTD[RbStats.columns].sort_values(by='RushTD', ascending=False)

,Player,RushAtt,RushYds,Y/A,RushTD,Tgt,Rec,RecYds,Y/R,RecTD,TotalTD,PPR,PosRank
5,Raheem Mostert*,209,1012,4.84,18,32,25,175,7.0,3,21,267.7,2
0,Christian McCaffrey*+,272,1459,5.36,14,83,67,564,8.42,7,21,391.3,1
53,Gus Edwards,198,810,4.09,13,13,12,180,15.0,0,13,187.0,14
32,David Montgomery,219,1015,4.63,13,24,16,117,7.31,0,13,207.2,9
12,Kyren Williams*,228,1144,5.02,12,48,32,206,6.44,3,15,255.0,4
15,Derrick Henry*,280,1167,4.17,12,36,28,214,7.64,0,12,246.7,5
11,Travis Etienne,267,1008,3.78,11,73,58,476,8.21,1,12,282.4,3
35,Jahmyr Gibbs*,182,945,5.19,10,71,52,316,6.08,1,11,242.1,10
125,Kareem Hunt,135,411,3.04,9,21,15,84,5.6,0,9,118.5,37
16,Joe Mixon,257,1034,4.02,9,64,52,376,7.23,3,12,267.0,6


### Elite Rushing TD

In [150]:
EliteRbRushTd = AvgRbRushTD[(AvgRbRushTD['RushTD']>12)]

In [151]:
EliteRbRushTd[RbStats.columns].sort_values(by='RushTD', ascending=False)

,Player,RushAtt,RushYds,Y/A,RushTD,Tgt,Rec,RecYds,Y/R,RecTD,TotalTD,PPR,PosRank
5,Raheem Mostert*,209,1012,4.84,18,32,25,175,7.0,3,21,267.7,2
0,Christian McCaffrey*+,272,1459,5.36,14,83,67,564,8.42,7,21,391.3,1
32,David Montgomery,219,1015,4.63,13,24,16,117,7.31,0,13,207.2,9
53,Gus Edwards,198,810,4.09,13,13,12,180,15.0,0,13,187.0,14


## Running Back Receptions

In [152]:
AvgRbRec[RbStats.columns].describe()

,RushAtt,RushYds,Y/A,RushTD,Tgt,Rec,RecYds,Y/R,RecTD,TotalTD,PPR,PosRank
count,23.0,23.0,23.0,23.0,23.0,23.0,23.0,23.0,23.0,23.0,23.0,23.0
mean,199.217391,838.73913,4.230435,5.304348,66.130435,52.434783,377.608696,7.140435,2.086957,7.391304,216.93913,21.391304
std,61.609442,278.651659,0.556282,3.197084,13.243538,10.714587,114.031789,1.375131,1.880837,4.196743,60.181844,13.600599
min,53.0,238.0,3.49,1.0,44.0,38.0,214.0,4.85,0.0,1.0,121.3,1.0
25%,179.5,668.0,3.825,3.5,57.0,44.0,295.5,6.075,1.0,5.0,180.85,10.5
50%,214.0,935.0,4.02,5.0,65.0,51.0,376.0,7.23,2.0,6.0,213.9,19.0
75%,242.5,999.5,4.56,6.0,73.5,58.0,460.5,8.305,3.5,9.0,244.2,31.5
max,272.0,1459.0,5.36,14.0,95.0,76.0,591.0,10.11,7.0,21.0,391.3,49.0


### Average Receptions

In [153]:
AvgRbRec[RbStats.columns].sort_values(by='Rec', ascending=False)

,Player,RushAtt,RushYds,Y/A,RushTD,Tgt,Rec,RecYds,Y/R,RecTD,TotalTD,PPR,PosRank
18,Breece Hall,223,994,4.46,5,95,76,591,7.78,4,9,290.5,7
88,Alvin Kamara,180,694,3.86,5,86,75,466,6.21,1,6,233.0,24
0,Christian McCaffrey*+,272,1459,5.36,14,83,67,564,8.42,7,21,391.3,1
23,Rachaad White,272,990,3.64,6,70,64,549,8.58,3,9,267.9,8
100,Jaylen Warren,149,784,5.26,4,74,61,370,6.07,0,4,196.4,30
38,Bijan Robinson,214,976,4.56,4,86,58,487,8.4,4,8,246.3,12
11,Travis Etienne,267,1008,3.78,11,73,58,476,8.21,1,12,282.4,3
64,Tony Pollard,252,1005,3.99,6,67,55,311,5.65,0,6,222.6,18
16,Joe Mixon,257,1034,4.02,9,64,52,376,7.23,3,12,267.0,6
35,Jahmyr Gibbs*,182,945,5.19,10,71,52,316,6.08,1,11,242.1,10


### Elite Receptions

In [154]:
EliteRbRec = AvgRbRec[(AvgRbRec['Rec']>58)]

In [155]:
EliteRbRec[RbStats.columns].sort_values(by='Rec', ascending=False)

,Player,RushAtt,RushYds,Y/A,RushTD,Tgt,Rec,RecYds,Y/R,RecTD,TotalTD,PPR,PosRank
18,Breece Hall,223,994,4.46,5,95,76,591,7.78,4,9,290.5,7
88,Alvin Kamara,180,694,3.86,5,86,75,466,6.21,1,6,233.0,24
0,Christian McCaffrey*+,272,1459,5.36,14,83,67,564,8.42,7,21,391.3,1
23,Rachaad White,272,990,3.64,6,70,64,549,8.58,3,9,267.9,8
100,Jaylen Warren,149,784,5.26,4,74,61,370,6.07,0,4,196.4,30


## Running Back Reception Yards

In [156]:
AvgRbRecYds[RbStats.columns].describe()

,RushAtt,RushYds,Y/A,RushTD,Tgt,Rec,RecYds,Y/R,RecTD,TotalTD,PPR,PosRank
count,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0
mean,197.4,827.5,4.22,5.45,68.35,53.6,409.6,7.7165,2.35,7.8,222.41,20.3
std,65.059811,290.921132,0.584817,3.363504,12.712509,11.090062,93.175217,1.308729,1.980829,4.360106,61.863967,13.989093
min,53.0,238.0,3.45,1.0,43.0,36.0,280.0,5.65,0.0,1.0,121.3,1.0
25%,178.75,681.0,3.84,3.75,59.75,47.0,318.25,6.675,1.0,5.75,184.325,9.5
50%,209.0,879.0,4.05,5.0,68.5,52.0,387.0,7.89,2.0,7.0,222.9,18.5
75%,248.25,996.75,4.5375,6.0,74.0,58.75,468.5,8.4525,4.0,9.25,251.475,30.25
max,272.0,1459.0,5.36,14.0,95.0,76.0,591.0,10.22,7.0,21.0,391.3,49.0


### Average Reception Yards

In [157]:
AvgRbRecYds[RbStats.columns].sort_values(by='RecYds', ascending=False)

,Player,RushAtt,RushYds,Y/A,RushTD,Tgt,Rec,RecYds,Y/R,RecTD,TotalTD,PPR,PosRank
18,Breece Hall,223,994,4.46,5,95,76,591,7.78,4,9,290.5,7
0,Christian McCaffrey*+,272,1459,5.36,14,83,67,564,8.42,7,21,391.3,1
23,Rachaad White,272,990,3.64,6,70,64,549,8.58,3,9,267.9,8
38,Bijan Robinson,214,976,4.56,4,86,58,487,8.4,4,8,246.3,12
11,Travis Etienne,267,1008,3.78,11,73,58,476,8.21,1,12,282.4,3
88,Alvin Kamara,180,694,3.86,5,86,75,466,6.21,1,6,233.0,24
174,Samaje Perine,53,238,4.49,1,56,50,455,9.1,0,1,121.3,49
37,James Cook*,237,1122,4.73,2,54,44,445,10.11,4,6,232.7,11
101,Austin Ekeler,179,628,3.51,5,74,51,436,8.55,1,6,185.4,31
152,Antonio Gibson,65,265,4.08,1,59,48,389,8.1,2,3,127.4,45


### Elite Reception Yards

In [158]:
EliteRbRecYds = AvgRbRecYds[(AvgRbRecYds['RecYds']>468)]

In [159]:
EliteRbRecYds[RbStats.columns].sort_values(by='RecYds', ascending=False)

,Player,RushAtt,RushYds,Y/A,RushTD,Tgt,Rec,RecYds,Y/R,RecTD,TotalTD,PPR,PosRank
18,Breece Hall,223,994,4.46,5,95,76,591,7.78,4,9,290.5,7
0,Christian McCaffrey*+,272,1459,5.36,14,83,67,564,8.42,7,21,391.3,1
23,Rachaad White,272,990,3.64,6,70,64,549,8.58,3,9,267.9,8
38,Bijan Robinson,214,976,4.56,4,86,58,487,8.4,4,8,246.3,12
11,Travis Etienne,267,1008,3.78,11,73,58,476,8.21,1,12,282.4,3


## Running Back Yards Per Reception

In [160]:
AvgRbYdsPerRec[RbStats.columns].describe()

,RushAtt,RushYds,Y/A,RushTD,Tgt,Rec,RecYds,Y/R,RecTD,TotalTD,PPR,PosRank
count,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0
mean,176.227273,733.090909,4.120455,5.272727,49.954545,39.181818,335.954545,8.856364,1.863636,7.136364,187.940909,26.045455
std,76.252046,339.324105,0.553332,4.037701,23.875625,18.538599,148.010449,1.668054,1.909761,4.601195,76.991661,17.008338
min,21.0,60.0,2.86,1.0,13.0,12.0,144.0,7.4,0.0,1.0,84.4,1.0
25%,112.0,499.0,3.6975,2.0,29.0,22.75,198.25,7.835,0.25,3.25,129.275,11.25
50%,182.5,737.0,4.125,4.5,48.5,36.5,332.0,8.41,1.0,6.0,183.25,26.5
75%,230.5,986.5,4.5125,6.75,70.0,51.75,452.5,9.0575,3.75,9.0,242.9,39.75
max,280.0,1459.0,5.36,14.0,95.0,76.0,591.0,15.0,7.0,21.0,391.3,56.0


### Average Yards/Reception

In [161]:
AvgRbYdsPerRec[RbStats.columns].sort_values(by='Y/R', ascending=False)

,Player,RushAtt,RushYds,Y/A,RushTD,Tgt,Rec,RecYds,Y/R,RecTD,TotalTD,PPR,PosRank
53,Gus Edwards,198,810,4.09,13,13,12,180,15.0,0,13,187.0,14
110,Tyler Allgeier,186,683,3.67,4,23,18,193,10.72,1,5,137.6,34
72,Brian Robinson Jr.,178,733,4.12,5,43,36,368,10.22,4,9,198.1,22
132,AJ Dillon,178,613,3.44,2,28,22,223,10.14,0,2,117.6,40
37,James Cook*,237,1122,4.73,2,54,44,445,10.11,4,6,232.7,11
174,Samaje Perine,53,238,4.49,1,56,50,455,9.1,0,1,121.3,49
59,Kenneth Walker III,219,905,4.13,8,37,29,259,8.93,1,9,199.4,16
23,Rachaad White,272,990,3.64,6,70,64,549,8.58,3,9,267.9,8
101,Austin Ekeler,179,628,3.51,5,74,51,436,8.55,1,6,185.4,31
160,Rico Dowdle,89,361,4.06,2,22,17,144,8.47,2,4,91.5,47


### Elite Yards/Reception

In [162]:
EliteRbYdsPerRec = AvgRbYdsPerRec[(AvgRbYdsPerRec['Y/R']>9)]

In [163]:
EliteRbYdsPerRec[RbStats.columns].sort_values(by='Y/R', ascending=False)

,Player,RushAtt,RushYds,Y/A,RushTD,Tgt,Rec,RecYds,Y/R,RecTD,TotalTD,PPR,PosRank
53,Gus Edwards,198,810,4.09,13,13,12,180,15.0,0,13,187.0,14
110,Tyler Allgeier,186,683,3.67,4,23,18,193,10.72,1,5,137.6,34
72,Brian Robinson Jr.,178,733,4.12,5,43,36,368,10.22,4,9,198.1,22
132,AJ Dillon,178,613,3.44,2,28,22,223,10.14,0,2,117.6,40
37,James Cook*,237,1122,4.73,2,54,44,445,10.11,4,6,232.7,11
174,Samaje Perine,53,238,4.49,1,56,50,455,9.1,0,1,121.3,49


## Running Back Receiving TD

In [164]:
AvgRbRecTD[RbStats.columns].describe()

,RushAtt,RushYds,Y/A,RushTD,Tgt,Rec,RecYds,Y/R,RecTD,TotalTD,PPR,PosRank
count,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0
mean,185.090909,819.090909,4.438182,5.590909,53.181818,41.5,308.0,7.324091,3.181818,8.772727,205.581818,21.909091
std,70.064534,321.732203,0.974121,4.510929,19.251806,15.506527,143.221108,1.400791,1.258736,5.032578,72.598725,16.352979
min,21.0,60.0,2.86,0.0,22.0,17.0,144.0,4.85,2.0,3.0,84.4,1.0
25%,178.5,708.25,3.915,2.25,37.0,27.75,192.0,6.41,2.0,5.0,170.825,8.75
50%,206.5,874.0,4.23,5.0,51.5,42.5,262.0,7.24,3.0,9.0,206.35,18.0
75%,226.75,1007.5,4.8125,7.0,63.75,50.25,385.75,8.325,4.0,9.75,252.825,32.75
max,272.0,1459.0,7.77,18.0,95.0,76.0,591.0,10.22,7.0,21.0,391.3,56.0


### Average Receiving TD

In [165]:
AvgRbRecTD[RbStats.columns].sort_values(by='RecTD', ascending=False)

,Player,RushAtt,RushYds,Y/A,RushTD,Tgt,Rec,RecYds,Y/R,RecTD,TotalTD,PPR,PosRank
0,Christian McCaffrey*+,272,1459,5.36,14,83,67,564,8.42,7,21,391.3,1
65,Jerome Ford,204,813,3.99,4,63,44,319,7.25,5,9,211.2,19
202,Jerick McKinnon,21,60,2.86,1,32,25,192,7.68,4,5,84.4,56
18,Breece Hall,223,994,4.46,5,95,76,591,7.78,4,9,290.5,7
37,James Cook*,237,1122,4.73,2,54,44,445,10.11,4,6,232.7,11
38,Bijan Robinson,214,976,4.56,4,86,58,487,8.4,4,8,246.3,12
43,Saquon Barkley,247,962,3.89,6,60,41,280,6.83,4,10,223.2,13
72,Brian Robinson Jr.,178,733,4.12,5,43,36,368,10.22,4,9,198.1,22
5,Raheem Mostert*,209,1012,4.84,18,32,25,175,7.0,3,21,267.7,2
126,Alexander Mattison,180,700,3.89,0,44,30,192,6.4,3,3,133.2,38


### Elite Receiving TD

In [166]:
EliteRbRecTD = AvgRbRecTD[(AvgRbRecTD['RecTD']>4)]

In [167]:
EliteRbRecTD[RbStats.columns].sort_values(by='RecTD', ascending=False)

,Player,RushAtt,RushYds,Y/A,RushTD,Tgt,Rec,RecYds,Y/R,RecTD,TotalTD,PPR,PosRank
0,Christian McCaffrey*+,272,1459,5.36,14,83,67,564,8.42,7,21,391.3,1
65,Jerome Ford,204,813,3.99,4,63,44,319,7.25,5,9,211.2,19


## Running Back Total TD

In [168]:
AvgRbTotalTD[RbStats.columns].describe()

,RushAtt,RushYds,Y/A,RushTD,Tgt,Rec,RecYds,Y/R,RecTD,TotalTD,PPR,PosRank
count,23.0,23.0,23.0,23.0,23.0,23.0,23.0,23.0,23.0,23.0,23.0,23.0
mean,214.217391,944.478261,4.48,8.73913,49.391304,38.173913,290.521739,7.691304,2.304348,11.043478,227.352174,13.73913
std,44.314944,196.566637,0.894559,3.633398,22.729115,17.98649,151.041797,1.963481,1.869243,3.710896,55.526897,9.269782
min,103.0,411.0,3.04,4.0,13.0,12.0,84.0,5.55,0.0,7.0,118.5,1.0
25%,190.5,811.5,4.04,6.0,34.5,27.0,177.5,6.635,1.0,9.0,196.8,6.5
50%,214.0,976.0,4.34,8.0,43.0,32.0,244.0,7.3,2.0,9.0,213.9,13.0
75%,251.0,1024.5,4.735,11.5,67.0,52.0,372.0,8.305,3.5,12.0,261.0,19.5
max,280.0,1459.0,7.77,18.0,95.0,76.0,591.0,15.0,7.0,21.0,391.3,37.0


### Average Total TD

In [169]:
AvgRbTotalTD[RbStats.columns].sort_values(by='TotalTD', ascending=False)

,Player,RushAtt,RushYds,Y/A,RushTD,Tgt,Rec,RecYds,Y/R,RecTD,TotalTD,PPR,PosRank
0,Christian McCaffrey*+,272,1459,5.36,14,83,67,564,8.42,7,21,391.3,1
5,Raheem Mostert*,209,1012,4.84,18,32,25,175,7.0,3,21,267.7,2
12,Kyren Williams*,228,1144,5.02,12,48,32,206,6.44,3,15,255.0,4
53,Gus Edwards,198,810,4.09,13,13,12,180,15.0,0,13,187.0,14
32,David Montgomery,219,1015,4.63,13,24,16,117,7.31,0,13,207.2,9
11,Travis Etienne,267,1008,3.78,11,73,58,476,8.21,1,12,282.4,3
15,Derrick Henry*,280,1167,4.17,12,36,28,214,7.64,0,12,246.7,5
16,Joe Mixon,257,1034,4.02,9,64,52,376,7.23,3,12,267.0,6
70,De'Von Achane,103,800,7.77,8,37,27,197,7.3,3,11,190.7,21
35,Jahmyr Gibbs*,182,945,5.19,10,71,52,316,6.08,1,11,242.1,10


### Elite Total TD

In [170]:
EliteRbTotalTD = AvgRbTotalTD[(AvgRbTotalTD['TotalTD']>12)]

In [171]:
EliteRbTotalTD[RbStats.columns].sort_values(by='TotalTD', ascending=False)

,Player,RushAtt,RushYds,Y/A,RushTD,Tgt,Rec,RecYds,Y/R,RecTD,TotalTD,PPR,PosRank
0,Christian McCaffrey*+,272,1459,5.36,14,83,67,564,8.42,7,21,391.3,1
5,Raheem Mostert*,209,1012,4.84,18,32,25,175,7.0,3,21,267.7,2
12,Kyren Williams*,228,1144,5.02,12,48,32,206,6.44,3,15,255.0,4
32,David Montgomery,219,1015,4.63,13,24,16,117,7.31,0,13,207.2,9
53,Gus Edwards,198,810,4.09,13,13,12,180,15.0,0,13,187.0,14


# Wide Receivers


In [236]:
# @title
fp.columns

Index(['Rk', 'Player', 'Tm', 'FantPos', 'Age', 'G', 'GS', 'Cmp', 'PassAtt',
       'PassYds', 'PassTD', 'Int', 'RushAtt', 'RushYds', 'Y/A', 'RushTD',
       'Tgt', 'Rec', 'RecYds', 'Y/R', 'RecTD', 'Fmb', 'FL', 'TotalTD', '2PM',
       '2PP', 'FantPt', 'PPR', 'DKPt', 'FDPt', 'VBD', 'PosRank', 'OvRank'],
      dtype='object')

In [237]:
# @title
WrStats = fp[['Player','Rec','RecYds','Y/R','RecTD','RushAtt','RushYds','Y/A','RushTD','TotalTD','PPR','PosRank']]

## Wide Receivers PPR

In [202]:
AvgWrRec = fp[(fp['Rec'] > fp['Rec'].mean()) & (Wr)]
AvgWrRecYds = fp[(fp['RecYds'] > fp['RecYds'].mean()) & (Wr)]
AvgWrYdsPerRec = fp[(fp['Y/R'] > fp['Y/R'].mean()) & (Wr) & (fp['PPR']>fp['PPR'].mean())]
AvgWrRecTD = fp[(fp['RecTD'] > fp['RecTD'].mean()) & (Wr)]
AvgWrRushAtt = fp[(fp['RushAtt'] > fp['RushAtt'].mean()) & (Wr)]
AvgWrRushYds = fp[(fp['RushYds'] > fp['RushYds'].mean()) & (Wr)]
AvgWrYdsPerAtt = fp[(fp['Y/A'] > fp['Y/A'].mean()) & (Wr) & (fp['PPR']>fp['PPR'].mean())]
AvgWrRushTD = fp[(fp['RushTD'] > fp['RushTD'].mean()) & (Wr)]
AvgWrTotalTD = fp[(fp['TotalTD'] > fp['TotalTD'].mean()) & (Wr)]


In [181]:
AvgWrPPR = fp[(fp['PPR'] > fp['PPR'].mean()) & (Wr)]

In [182]:
AvgWrPPR.describe()

,Age,G,GS,Cmp,PassAtt,PassYds,PassTD,Int,RushAtt,RushYds,...,TotalTD,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank
count,81.0,81.0,81.0,81.0,81.0,81.0,81.0,81.0,81.0,81.0,...,81.0,19.0,1.0,81.0,81.0,81.0,81.0,29.0,81.0,29.0
mean,25.802469,15.358025,12.37037,0.08642,0.197531,1.098765,0.024691,0.012346,3.160494,18.555556,...,5.135802,1.105263,1.0,114.246914,177.025926,180.630864,145.64321,36.758621,41.246914,37.896552
std,3.047211,2.204023,4.084864,0.323942,0.534316,6.301994,0.15615,0.111111,5.812609,39.031398,...,3.093352,0.315302,<NA>,49.374976,72.602827,72.886474,60.740367,31.922512,23.974221,21.883368
min,21.0,8.0,3.0,0.0,0.0,-7.0,0.0,0.0,0.0,-12.0,...,0.0,1.0,1.0,42.0,84.8,87.8,63.3,1.0,1.0,2.0
25%,23.0,15.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,1.0,1.0,73.0,113.1,116.1,93.6,16.0,21.0,21.0
50%,26.0,16.0,13.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,5.0,1.0,1.0,105.0,161.4,165.4,134.9,28.0,41.0,39.0
75%,28.0,17.0,16.0,0.0,0.0,0.0,0.0,0.0,4.0,22.0,...,7.0,1.0,1.0,148.0,225.4,228.4,186.3,51.0,61.0,54.0
max,33.0,17.0,17.0,2.0,3.0,49.0,1.0,1.0,37.0,225.0,...,14.0,2.0,1.0,268.0,403.2,411.2,335.7,131.0,92.0,75.0


### Average PPR

In [183]:
AvgWrPPR[WrStats.columns].sort_values(by='PPR', ascending=False)

,Player,Rec,RecYds,Y/R,RecTD,RushAtt,RushYds,Y/A,RushTD,TotalTD,PPR,PosRank
1,CeeDee Lamb*+,135,1749,12.96,12,14,113,8.07,2,14,403.2,1
3,Tyreek Hill*+,119,1799,15.12,13,6,15,2.5,0,13,376.4,2
6,Amon-Ra St. Brown*+,119,1515,12.73,10,4,24,6.0,0,10,330.9,3
13,Puka Nacua*,105,1486,14.15,6,12,89,7.42,0,6,298.5,5
19,A.J. Brown*,106,1456,13.74,7,0,0,<NA>,0,7,289.6,8
...,...,...,...,...,...,...,...,...,...,...,...,...
195,Justin Watson,27,460,17.04,3,0,0,<NA>,0,3,89.0,72
211,Michael Gallup,34,418,12.29,2,0,0,<NA>,0,2,87.8,80
198,Demarcus Robinson,26,371,14.27,4,1,23,23.0,0,4,87.4,73
208,Nick Westbrook-Ikhine,28,370,13.21,3,0,0,<NA>,0,3,85.0,75


### Elite PPR

In [184]:
EliteWrPPR = AvgWrPPR[(AvgWrPPR['PPR']>225)]

In [185]:
EliteWrPPR[WrStats.columns].sort_values(by='PPR', ascending=False)

,Player,Rec,RecYds,Y/R,RecTD,RushAtt,RushYds,Y/A,RushTD,TotalTD,PPR,PosRank
1,CeeDee Lamb*+,135,1749,12.96,12,14,113,8.07,2,14,403.2,1
3,Tyreek Hill*+,119,1799,15.12,13,6,15,2.5,0,13,376.4,2
6,Amon-Ra St. Brown*+,119,1515,12.73,10,4,24,6.0,0,10,330.9,3
13,Puka Nacua*,105,1486,14.15,6,12,89,7.42,0,6,298.5,5
19,A.J. Brown*,106,1456,13.74,7,0,0,<NA>,0,7,289.6,8
14,D.J. Moore,96,1364,14.21,8,4,21,5.25,1,9,286.5,6
8,Mike Evans*,79,1255,15.89,13,0,0,<NA>,0,13,282.5,4
26,Keenan Allen*,108,1243,11.51,7,2,6,3.0,0,7,278.9,11
28,Stefon Diggs*,107,1183,11.06,8,1,5,5.0,0,8,273.8,12
34,Davante Adams,103,1144,11.11,8,0,0,<NA>,0,8,265.4,14


## Wide Receiver Receptions

In [186]:
AvgWrRec[WrStats.columns].describe()

,Rec,RecYds,Y/R,RecTD,RushAtt,RushYds,Y/A,RushTD,TotalTD,PPR,PosRank
count,113.0,113.0,113.0,113.0,113.0,113.0,64.0,113.0,113.0,113.0,113.0
mean,51.79646,666.292035,12.82708,3.823009,2.79646,16.584071,5.494688,0.176991,4.035398,143.99292,59.026549
std,27.9975,380.197004,2.627249,3.03912,5.220426,34.701978,5.153272,0.615718,3.204152,81.208989,36.381525
min,19.0,104.0,5.2,0.0,0.0,-12.0,-12.0,0.0,0.0,30.4,1.0
25%,28.0,370.0,11.06,1.0,0.0,0.0,3.0,0.0,2.0,76.5,29.0
50%,43.0,574.0,12.68,3.0,1.0,1.0,5.535,0.0,3.0,120.0,57.0
75%,70.0,911.0,14.9,5.0,3.0,21.0,7.4825,0.0,6.0,209.2,86.0
max,135.0,1799.0,19.27,13.0,37.0,225.0,23.0,5.0,14.0,403.2,161.0


### Average Receptions

In [187]:
AvgWrRec[WrStats.columns].sort_values(by='Rec', ascending=False)

,Player,Rec,RecYds,Y/R,RecTD,RushAtt,RushYds,Y/A,RushTD,TotalTD,PPR,PosRank
1,CeeDee Lamb*+,135,1749,12.96,12,14,113,8.07,2,14,403.2,1
6,Amon-Ra St. Brown*+,119,1515,12.73,10,4,24,6.0,0,10,330.9,3
3,Tyreek Hill*+,119,1799,15.12,13,6,15,2.5,0,13,376.4,2
63,Michael Pittman Jr.,109,1152,10.57,4,0,0,<NA>,0,4,250.2,25
26,Keenan Allen*,108,1243,11.51,7,2,6,3.0,0,7,278.9,11
...,...,...,...,...,...,...,...,...,...,...,...,...
274,Jauan Jennings,19,265,13.95,1,0,0,<NA>,0,1,51.5,109
282,Mike Williams,19,249,13.11,1,1,3,3.0,0,1,50.2,111
218,Tre Tucker,19,331,17.42,2,10,77,7.7,0,2,71.8,81
212,Brandon Johnson,19,284,14.95,4,0,0,<NA>,0,4,73.4,79


### Elite Receptions

In [188]:
EliteWrRec = AvgWrRec[(AvgWrRec['Rec']>70)]

In [235]:
EliteWrRec[WrStats.columns].sort_values(by='Rec', ascending=False)

,Player,Rec,RecYds,Y/R,RecTD,RushAtt,RushYds,Y/A,RushTD,TotalTD,PPR,PosRank
1,CeeDee Lamb*+,135,1749,12.96,12,14,113,8.07,2,14,403.2,1
6,Amon-Ra St. Brown*+,119,1515,12.73,10,4,24,6.0,0,10,330.9,3
3,Tyreek Hill*+,119,1799,15.12,13,6,15,2.5,0,13,376.4,2
63,Michael Pittman Jr.,109,1152,10.57,4,0,0,<NA>,0,4,250.2,25
26,Keenan Allen*,108,1243,11.51,7,2,6,3.0,0,7,278.9,11
28,Stefon Diggs*,107,1183,11.06,8,1,5,5.0,0,8,273.8,12
19,A.J. Brown*,106,1456,13.74,7,0,0,<NA>,0,7,289.6,8
13,Puka Nacua*,105,1486,14.15,6,12,89,7.42,0,6,298.5,5
34,Davante Adams,103,1144,11.11,8,0,0,<NA>,0,8,265.4,14
104,Adam Thielen,103,1014,9.84,4,1,6,6.0,0,4,231.0,31


## Wide Receiver Reception Yards

In [190]:
AvgWrRecYds[WrStats.columns].describe()

,Rec,RecYds,Y/R,RecTD,RushAtt,RushYds,Y/A,RushTD,TotalTD,PPR,PosRank
count,116.0,116.0,116.0,116.0,116.0,116.0,66.0,116.0,116.0,116.0,116.0
mean,50.637931,657.827586,13.18431,3.758621,2.732759,16.181034,5.39303,0.163793,3.956897,141.467241,59.87069
std,28.481389,378.083273,2.483695,3.037793,5.106446,34.3105,5.16045,0.603525,3.207036,81.629452,35.773084
min,12.0,209.0,8.24,0.0,0.0,-12.0,-12.0,0.0,0.0,37.2,1.0
25%,27.0,353.5,11.25,1.0,0.0,0.0,3.0,0.0,1.75,72.25,29.75
50%,41.0,566.0,13.095,3.0,1.0,0.0,5.535,0.0,3.0,117.65,58.5
75%,69.25,906.5,15.0,5.0,3.25,18.75,7.3975,0.0,5.25,208.9,88.25
max,135.0,1799.0,20.5,13.0,37.0,225.0,23.0,5.0,14.0,403.2,132.0


### Average Reception Yards

In [191]:
AvgWrRecYds[WrStats.columns].sort_values(by='RecYds', ascending=False)

,Player,Rec,RecYds,Y/R,RecTD,RushAtt,RushYds,Y/A,RushTD,TotalTD,PPR,PosRank
3,Tyreek Hill*+,119,1799,15.12,13,6,15,2.5,0,13,376.4,2
1,CeeDee Lamb*+,135,1749,12.96,12,14,113,8.07,2,14,403.2,1
6,Amon-Ra St. Brown*+,119,1515,12.73,10,4,24,6.0,0,10,330.9,3
13,Puka Nacua*,105,1486,14.15,6,12,89,7.42,0,6,298.5,5
19,A.J. Brown*,106,1456,13.74,7,0,0,<NA>,0,7,289.6,8
...,...,...,...,...,...,...,...,...,...,...,...,...
299,Jamal Agnew,14,225,16.07,1,4,-2,-0.5,0,1,40.3,118
333,Cedric Tillman,21,224,10.67,0,1,8,8.0,0,0,44.2,128
321,Treylon Burks,16,221,13.81,0,5,18,3.6,0,0,39.9,125
284,Bo Melton,16,218,13.63,1,3,21,7.0,0,1,45.9,113


### Elite Reception Yards

In [192]:
EliteWrRecYds = AvgWrRecYds[(AvgWrRecYds['RecYds']>906)]

In [193]:
EliteWrRecYds[WrStats.columns].sort_values(by='RecYds', ascending=False)

,Player,Rec,RecYds,Y/R,RecTD,RushAtt,RushYds,Y/A,RushTD,TotalTD,PPR,PosRank
3,Tyreek Hill*+,119,1799,15.12,13,6,15,2.5,0,13,376.4,2
1,CeeDee Lamb*+,135,1749,12.96,12,14,113,8.07,2,14,403.2,1
6,Amon-Ra St. Brown*+,119,1515,12.73,10,4,24,6.0,0,10,330.9,3
13,Puka Nacua*,105,1486,14.15,6,12,89,7.42,0,6,298.5,5
19,A.J. Brown*,106,1456,13.74,7,0,0,<NA>,0,7,289.6,8
14,D.J. Moore,96,1364,14.21,8,4,21,5.25,1,9,286.5,6
24,Brandon Aiyuk,75,1342,17.89,7,0,0,<NA>,0,7,249.2,10
22,Nico Collins,80,1297,16.21,8,1,7,7.0,0,8,260.4,9
8,Mike Evans*,79,1255,15.89,13,0,0,<NA>,0,13,282.5,4
42,Amari Cooper*,72,1250,17.36,5,0,0,<NA>,0,5,227.0,16


## Wide Receiver Yards/Reception


In [207]:
AvgWrYdsPerRec[WrStats.columns].describe()

,Rec,RecYds,Y/R,RecTD,RushAtt,RushYds,Y/A,RushTD,TotalTD,PPR,PosRank
count,78.0,78.0,78.0,78.0,78.0,78.0,46.0,78.0,78.0,78.0,78.0
mean,63.371795,832.769231,13.328718,5.064103,2.807692,15.871795,4.75087,0.192308,5.282051,179.69359,40.012821
std,25.878893,340.977698,2.187084,2.820794,5.137228,34.351642,5.08617,0.703566,3.053197,72.571012,23.419657
min,26.0,370.0,9.84,0.0,0.0,-12.0,-12.0,0.0,0.0,85.0,1.0
25%,39.25,565.5,11.3475,3.0,0.0,0.0,2.6075,0.0,3.0,114.75,20.25
50%,59.5,764.0,13.1,5.0,1.0,0.5,5.395,0.0,5.0,164.7,39.5
75%,79.0,1063.75,15.085,7.0,4.0,20.25,7.0,0.0,7.0,226.6,58.75
max,135.0,1799.0,18.1,13.0,37.0,225.0,23.0,5.0,14.0,403.2,83.0


### Average Yards/Reception

In [208]:
AvgWrYdsPerRec[WrStats.columns].sort_values(by='Y/R', ascending=False)

,Player,Rec,RecYds,Y/R,RecTD,RushAtt,RushYds,Y/A,RushTD,TotalTD,PPR,PosRank
55,George Pickens,63,1140,18.1,5,3,18,6.0,0,5,208.8,23
24,Brandon Aiyuk,75,1342,17.89,7,0,0,<NA>,0,7,249.2,10
42,Amari Cooper*,72,1250,17.36,5,0,0,<NA>,0,5,227.0,16
177,Noah Brown,33,567,17.18,2,1,-1,-1.0,0,2,101.6,66
195,Justin Watson,27,460,17.04,3,0,0,<NA>,0,3,89.0,72
...,...,...,...,...,...,...,...,...,...,...,...,...
141,Jaxon Smith-Njigba,63,628,9.97,4,0,0,<NA>,0,4,149.8,51
154,Tyler Boyd,67,667,9.96,2,2,11,5.5,0,2,144.8,55
134,Curtis Samuel,62,613,9.89,4,7,39,5.57,1,5,157.2,46
207,Trey Palmer,39,385,9.87,3,3,22,7.33,0,3,95.7,76


### Elite Yards/Reception

In [205]:
EliteWrYdsPerRec = AvgWrYdsPerRec[(AvgWrYdsPerRec['Y/R']>15)]

In [206]:
EliteWrYdsPerRec[WrStats.columns].sort_values(by='Y/R', ascending=False)

,Player,Rec,RecYds,Y/R,RecTD,RushAtt,RushYds,Y/A,RushTD,TotalTD,PPR,PosRank
55,George Pickens,63,1140,18.1,5,3,18,6.0,0,5,208.8,23
24,Brandon Aiyuk,75,1342,17.89,7,0,0,<NA>,0,7,249.2,10
42,Amari Cooper*,72,1250,17.36,5,0,0,<NA>,0,5,227.0,16
177,Noah Brown,33,567,17.18,2,1,-1,-1.0,0,2,101.6,66
195,Justin Watson,27,460,17.04,3,0,0,<NA>,0,3,89.0,72
39,D.K. Metcalf*,66,1114,16.88,8,0,0,<NA>,0,8,225.4,15
115,Gabriel Davis,45,746,16.58,7,1,-2,-2.0,0,7,161.4,38
22,Nico Collins,80,1297,16.21,8,1,7,7.0,0,8,260.4,9
165,Odell Beckham Jr.,35,565,16.14,3,0,0,<NA>,0,3,107.5,63
190,Alec Pierce,32,514,16.06,2,0,0,<NA>,0,2,95.4,68


## Wide Receiver Receiving TD

In [209]:
AvgWrRecTD[WrStats.columns].describe()

,Rec,RecYds,Y/R,RecTD,RushAtt,RushYds,Y/A,RushTD,TotalTD,PPR,PosRank
count,93.0,93.0,93.0,93.0,93.0,93.0,48.0,93.0,93.0,93.0,93.0
mean,54.88172,720.526882,13.20172,4.763441,2.462366,14.397849,4.813958,0.193548,4.978495,158.026882,50.612903
std,30.264001,401.089836,2.521965,2.651488,4.891339,33.424522,5.025006,0.663621,2.855119,82.637798,32.683938
min,6.0,68.0,6.73,2.0,0.0,-12.0,-12.0,0.0,2.0,30.8,1.0
25%,33.0,406.0,11.32,3.0,0.0,0.0,2.7025,0.0,3.0,94.0,24.0
50%,51.0,667.0,13.01,4.0,1.0,0.0,5.395,0.0,4.0,150.3,47.0
75%,75.0,1016.0,15.07,7.0,3.0,12.0,7.0,0.0,7.0,218.6,73.0
max,135.0,1799.0,20.5,13.0,37.0,225.0,23.0,5.0,14.0,403.2,129.0


### Average Receiving TD

In [210]:
AvgWrRecTD[WrStats.columns].sort_values(by='RecTD', ascending=False)

,Player,Rec,RecYds,Y/R,RecTD,RushAtt,RushYds,Y/A,RushTD,TotalTD,PPR,PosRank
8,Mike Evans*,79,1255,15.89,13,0,0,<NA>,0,13,282.5,4
3,Tyreek Hill*+,119,1799,15.12,13,6,15,2.5,0,13,376.4,2
1,CeeDee Lamb*+,135,1749,12.96,12,14,113,8.07,2,14,403.2,1
6,Amon-Ra St. Brown*+,119,1515,12.73,10,4,24,6.0,0,10,330.9,3
75,Courtland Sutton,59,772,13.08,10,0,0,<NA>,0,10,190.2,28
...,...,...,...,...,...,...,...,...,...,...,...,...
210,Jameson Williams,24,354,14.75,2,3,29,9.67,1,3,80.3,77
209,Quentin Johnston,38,431,11.34,2,3,9,3.0,0,2,94.0,78
167,Elijah Moore,59,640,10.85,2,9,11,1.22,0,2,132.1,62
177,Noah Brown,33,567,17.18,2,1,-1,-1.0,0,2,101.6,66


### Elite Receiving TD

In [211]:
EliteWrRecTd = AvgWrRecTD[(AvgWrRecTD['RecTD']>7)]

In [212]:
EliteWrRecTd[WrStats.columns].sort_values(by='RecTD', ascending=False)

,Player,Rec,RecYds,Y/R,RecTD,RushAtt,RushYds,Y/A,RushTD,TotalTD,PPR,PosRank
3,Tyreek Hill*+,119,1799,15.12,13,6,15,2.5,0,13,376.4,2
8,Mike Evans*,79,1255,15.89,13,0,0,<NA>,0,13,282.5,4
1,CeeDee Lamb*+,135,1749,12.96,12,14,113,8.07,2,14,403.2,1
6,Amon-Ra St. Brown*+,119,1515,12.73,10,4,24,6.0,0,10,330.9,3
47,Jordan Addison,70,911,13.01,10,1,2,2.0,0,10,221.3,19
75,Courtland Sutton,59,772,13.08,10,0,0,<NA>,0,10,190.2,28
14,D.J. Moore,96,1364,14.21,8,4,21,5.25,1,9,286.5,6
22,Nico Collins,80,1297,16.21,8,1,7,7.0,0,8,260.4,9
28,Stefon Diggs*,107,1183,11.06,8,1,5,5.0,0,8,273.8,12
34,Davante Adams,103,1144,11.11,8,0,0,<NA>,0,8,265.4,14


## Wide Receiver Rushing Attempts

In [213]:
AvgWrRushAtt[WrStats.columns].describe()

,Rec,RecYds,Y/R,RecTD,RushAtt,RushYds,Y/A,RushTD,TotalTD,PPR,PosRank
count,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
mean,50.0,622.0,11.835,4.0,32.5,201.5,6.22,3.0,7.0,174.35,37.0
std,14.142136,381.837662,4.292138,4.242641,6.363961,33.234019,0.19799,2.828427,7.071068,98.075711,42.426407
min,40.0,352.0,8.8,1.0,28.0,178.0,6.08,1.0,2.0,105.0,7.0
25%,45.0,487.0,10.3175,2.5,30.25,189.75,6.15,2.0,4.5,139.675,22.0
50%,50.0,622.0,11.835,4.0,32.5,201.5,6.22,3.0,7.0,174.35,37.0
75%,55.0,757.0,13.3525,5.5,34.75,213.25,6.29,4.0,9.5,209.025,52.0
max,60.0,892.0,14.87,7.0,37.0,225.0,6.36,5.0,12.0,243.7,67.0


### Average Rushing Attempts

In [214]:
AvgWrRushAtt[WrStats.columns].sort_values(by='RushAtt', ascending=False)

,Player,Rec,RecYds,Y/R,RecTD,RushAtt,RushYds,Y/A,RushTD,TotalTD,PPR,PosRank
20,Deebo Samuel,60,892,14.87,7,37,225,6.08,5,12,243.7,7
187,Rondale Moore,40,352,8.8,1,28,178,6.36,1,2,105.0,67


### Elite Rushing Attempts

In [215]:
EliteWrRushAtt = AvgWrRushAtt[(AvgWrRushAtt['RushAtt']>34)]

In [216]:
EliteWrRushAtt[WrStats.columns].sort_values(by='RushAtt', ascending=False)

,Player,Rec,RecYds,Y/R,RecTD,RushAtt,RushYds,Y/A,RushTD,TotalTD,PPR,PosRank
20,Deebo Samuel,60,892,14.87,7,37,225,6.08,5,12,243.7,7


## Wide Receiver Rushing Yards

In [217]:
AvgWrRushYds[WrStats.columns].describe()

,Rec,RecYds,Y/R,RecTD,RushAtt,RushYds,Y/A,RushTD,TotalTD,PPR,PosRank
count,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0
mean,54.333333,663.166667,10.666667,5.166667,19.166667,141.0,8.09,1.833333,7.166667,177.416667,52.166667
std,45.116146,630.739698,3.70568,4.622409,10.796604,49.505555,1.944675,1.722401,5.528713,138.664565,52.522059
min,12.0,66.0,4.4,0.0,11.0,101.0,6.08,0.0,1.0,35.7,1.0
25%,21.25,183.25,9.245,1.5,11.75,110.75,6.5725,1.0,2.5,71.025,9.75
50%,50.0,572.5,11.485,5.0,14.0,116.0,7.64,1.5,7.0,161.1,42.5
75%,63.0,867.25,12.8175,7.75,24.5,163.25,9.5175,2.0,11.5,237.075,82.0
max,135.0,1749.0,14.87,12.0,37.0,225.0,10.82,5.0,14.0,403.2,133.0


### Average Rushing Yards

In [218]:
AvgWrRushYds[WrStats.columns].sort_values(by='RushYds', ascending=False)

,Player,Rec,RecYds,Y/R,RecTD,RushAtt,RushYds,Y/A,RushTD,TotalTD,PPR,PosRank
20,Deebo Samuel,60,892,14.87,7,37,225,6.08,5,12,243.7,7
187,Rondale Moore,40,352,8.8,1,28,178,6.36,1,2,105.0,67
45,Jayden Reed,64,793,12.39,8,11,119,10.82,2,10,217.2,18
1,CeeDee Lamb*+,135,1749,12.96,12,14,113,8.07,2,14,403.2,1
227,KaVontae Turpin,12,127,10.58,3,11,110,10.0,1,4,59.7,87
343,Derius Davis,15,66,4.4,0,14,101,7.21,0,1,35.7,133


### Elite Rushing Yards

In [219]:
EliteWrRushYds = AvgWrRushYds[(AvgWrRushYds['RushYds']>163)]

In [220]:
EliteWrRushYds[WrStats.columns].sort_values(by='RushYds', ascending=False)

,Player,Rec,RecYds,Y/R,RecTD,RushAtt,RushYds,Y/A,RushTD,TotalTD,PPR,PosRank
20,Deebo Samuel,60,892,14.87,7,37,225,6.08,5,12,243.7,7
187,Rondale Moore,40,352,8.8,1,28,178,6.36,1,2,105.0,67


## Wide Receiver Yards/Rush Attempt

In [221]:
AvgWrYdsPerAtt[WrStats.columns].describe()

,Rec,RecYds,Y/R,RecTD,RushAtt,RushYds,Y/A,RushTD,TotalTD,PPR,PosRank
count,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0
mean,65.387097,828.677419,12.72129,4.806452,6.612903,46.322581,7.469032,0.548387,5.387097,185.16129,39.258065
std,27.219696,370.042645,2.348527,3.059623,7.931698,52.206249,3.45972,1.059519,3.537206,79.591945,25.238816
min,26.0,352.0,8.75,0.0,1.0,5.0,4.5,0.0,0.0,87.4,1.0
25%,43.0,567.5,11.1,2.0,1.5,10.5,5.785,0.0,2.5,112.75,19.0
50%,60.0,719.0,12.39,4.0,4.0,24.0,6.2,0.0,5.0,165.0,37.0
75%,78.5,1040.5,14.24,7.5,8.0,53.5,7.745,1.0,8.0,227.3,60.0
max,135.0,1749.0,18.1,12.0,37.0,225.0,23.0,5.0,14.0,403.2,83.0


### Average Yards/Rush Attempt

In [222]:
AvgWrYdsPerAtt[WrStats.columns].sort_values(by='Y/A', ascending=False)

,Player,Rec,RecYds,Y/R,RecTD,RushAtt,RushYds,Y/A,RushTD,TotalTD,PPR,PosRank
198,Demarcus Robinson,26,371,14.27,4,1,23,23.0,0,4,87.4,73
144,Marquise Brown,51,574,11.25,4,2,23,11.5,0,4,134.7,52
45,Jayden Reed,64,793,12.39,8,11,119,10.82,2,10,217.2,18
194,Kalif Raymond,35,489,13.97,1,7,75,10.71,0,1,97.4,70
162,Khalil Shakir,39,611,15.67,2,1,10,10.0,0,2,113.1,58
169,Wan'Dale Robinson,60,525,8.75,1,9,87,9.67,1,2,133.2,61
106,Chris Godwin,83,1024,12.34,2,4,38,9.5,1,3,209.2,33
1,CeeDee Lamb*+,135,1749,12.96,12,14,113,8.07,2,14,403.2,1
13,Puka Nacua*,105,1486,14.15,6,12,89,7.42,0,6,298.5,5
207,Trey Palmer,39,385,9.87,3,3,22,7.33,0,3,95.7,76


### Elite Yards/Rush Attempt

In [223]:
EliteWrYdsPerAtt = AvgWrYdsPerAtt[(AvgWrYdsPerAtt['Y/A']>7.7)]

In [224]:
EliteWrYdsPerAtt[WrStats.columns].sort_values(by='Y/A', ascending=False)

,Player,Rec,RecYds,Y/R,RecTD,RushAtt,RushYds,Y/A,RushTD,TotalTD,PPR,PosRank
198,Demarcus Robinson,26,371,14.27,4,1,23,23.0,0,4,87.4,73
144,Marquise Brown,51,574,11.25,4,2,23,11.5,0,4,134.7,52
45,Jayden Reed,64,793,12.39,8,11,119,10.82,2,10,217.2,18
194,Kalif Raymond,35,489,13.97,1,7,75,10.71,0,1,97.4,70
162,Khalil Shakir,39,611,15.67,2,1,10,10.0,0,2,113.1,58
169,Wan'Dale Robinson,60,525,8.75,1,9,87,9.67,1,2,133.2,61
106,Chris Godwin,83,1024,12.34,2,4,38,9.5,1,3,209.2,33
1,CeeDee Lamb*+,135,1749,12.96,12,14,113,8.07,2,14,403.2,1


## Wide Receiver Rushing TD

In [225]:
AvgWrRushTD[WrStats.columns].describe()

,Rec,RecYds,Y/R,RecTD,RushAtt,RushYds,Y/A,RushTD,TotalTD,PPR,PosRank
count,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0
mean,53.0625,632.125,11.265,4.0,10.5,77.5625,7.495,1.4375,5.5625,157.5625,54.75
std,35.463538,478.141384,2.298428,3.596294,9.409215,59.983296,2.240729,1.030776,4.081973,103.975195,39.750052
min,8.0,51.0,6.38,0.0,1.0,3.0,3.0,1.0,2.0,32.5,1.0
25%,20.5,220.75,10.2125,1.0,4.0,35.75,5.8925,1.0,2.0,59.45,20.25
50%,60.0,569.0,11.02,2.5,8.0,62.5,7.535,1.0,3.5,145.2,53.5
75%,72.5,866.5,12.5325,7.25,11.0,110.75,9.5425,1.25,9.25,217.55,89.5
max,135.0,1749.0,14.87,12.0,37.0,225.0,10.82,5.0,14.0,403.2,124.0


### Average Rushing TD

In [227]:
AvgWrRushTD[WrStats.columns].sort_values(by='RushTD', ascending=False)

,Player,Rec,RecYds,Y/R,RecTD,RushAtt,RushYds,Y/A,RushTD,TotalTD,PPR,PosRank
20,Deebo Samuel,60,892,14.87,7,37,225,6.08,5,12,243.7,7
1,CeeDee Lamb*+,135,1749,12.96,12,14,113,8.07,2,14,403.2,1
45,Jayden Reed,64,793,12.39,8,11,119,10.82,2,10,217.2,18
51,Jakobi Meyers,71,807,11.37,8,4,24,6.0,2,10,218.6,21
14,D.J. Moore,96,1364,14.21,8,4,21,5.25,1,9,286.5,6
103,Zay Flowers,77,858,11.14,5,8,56,7.0,1,6,206.4,30
106,Chris Godwin,83,1024,12.34,2,4,38,9.5,1,3,209.2,33
134,Curtis Samuel,62,613,9.89,4,7,39,5.57,1,5,157.2,46
169,Wan'Dale Robinson,60,525,8.75,1,9,87,9.67,1,2,133.2,61
187,Rondale Moore,40,352,8.8,1,28,178,6.36,1,2,105.0,67


### Elite Rushing TD

In [228]:
EliteWrRushTD = AvgWrRushTD[(AvgWrRushTD['RushTD']>1)]

In [230]:
EliteWrRushTD[WrStats.columns].sort_values(by='RushTD', ascending=False)

,Player,Rec,RecYds,Y/R,RecTD,RushAtt,RushYds,Y/A,RushTD,TotalTD,PPR,PosRank
20,Deebo Samuel,60,892,14.87,7,37,225,6.08,5,12,243.7,7
1,CeeDee Lamb*+,135,1749,12.96,12,14,113,8.07,2,14,403.2,1
45,Jayden Reed,64,793,12.39,8,11,119,10.82,2,10,217.2,18
51,Jakobi Meyers,71,807,11.37,8,4,24,6.0,2,10,218.6,21


## Wide Receiver Total TD

### Average Total TD

### Elite Total TD

In [233]:
EliteWrTotalTD = AvgWrTotalTD[(AvgWrTotalTD['TotalTD']>7)]